# Occupational Fit — Preprocessing Pipeline

## Objective
Construct a measure of the quality of matches between jobs and workers by:
1. **Title→SOC mapping**: Mapping `jobtitle_raw` to O*NET-SOC codes via exact + fuzzy match against Alternate Titles / Reported Titles
2. **Task/skill similarity**: TF-IDF (char n-grams) between user profile docs and SOC task descriptions
3. **Requirement matching**: Education, experience, and training gap scores using O*NET scales (RL, RW, PT)
4. **Composite score**: Weighted blend of the above components

## Key Assumptions
- **Exact title matches are trusted**; fuzzy matches require score ≥ 90 (English) or ≥ 97 (non-English)
- **Education level** is mapped to O*NET Required Level (RL) scale 1–12 from `degree_raw` text via regex heuristics
- **Experience** is measured as cumulative prior work duration (years), converted to O*NET Related Work (RW) categories 1–11
- **Training requirement** is approximated using the same experience proxy (user_exp_level vs req_train_level) — this is a known weak proxy; a better signal would come from certifications or formal training records, which are not in the current data
- **Missing sub-scores** are imputed at 0.5 (neutral) — this dampens the score for users with sparse data but avoids penalizing them unfairly
- **Composite weights** (task 0.60, edu 0.20, exp 0.15, training 0.05) are hand-tuned; the high task weight reflects that task alignment is the strongest signal for occupational fit. Ideally these would be calibrated on labeled outcomes (retention, transitions)
- **Non-English titles** get a stricter fuzzy threshold because character-level fuzzy similarity across languages is unreliable

In [1]:
# Step 1: Open the ZIP, extract it, and load the first (core) file: individual_position.csv
# Assumes the zip is at: /mnt/data/sample_data.zip

from pathlib import Path
import zipfile
import pandas as pd

ZIP_PATH = Path("../data/sample_data.zip")
OUT_DIR = Path("../data/sample_data_extracted")

def extract_zip(zip_path: Path, out_dir: Path) -> list[Path]:
    out_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(out_dir)
        extracted = [out_dir / name for name in z.namelist()]
    # Return only real files (not directory entries)
    return [p for p in extracted if p.exists() and p.is_file()]

def list_files(root: Path) -> list[Path]:
    return sorted([p for p in root.rglob("*") if p.is_file()])

# 1) Extract
if not ZIP_PATH.exists():
    raise FileNotFoundError(f"ZIP not found at {ZIP_PATH}")

extracted_files = extract_zip(ZIP_PATH, OUT_DIR)

print(f"Extracted to: {OUT_DIR}")
print("Top-level extracted files:")
for p in sorted(set([f for f in extracted_files])):
    print(" -", p.relative_to(OUT_DIR))

# 2) Find the first file we want to build on: individual_position.csv
all_files = list_files(OUT_DIR)
pos_path = next((p for p in all_files if p.name.lower() == "individual_position.csv"), None)

if pos_path is None:
    raise FileNotFoundError("Could not find individual_position.csv inside the extracted zip.")

print("\nLoading:", pos_path)

# 3) Load it
df_pos = pd.read_csv(pos_path)

print("\nShape:", df_pos.shape)
print("\nColumns:", list(df_pos.columns))

# 4) Quick peek (first rows)
display(df_pos.head(10))

# 5) Minimal hygiene: parse dates if they exist, and standardize text fields
date_candidates = ["start_date", "end_date", "date_start", "date_end", "start", "end"]
for col in df_pos.columns:
    if col.lower() in date_candidates or "date" in col.lower():
        # attempt parse, but keep original if parsing fails widely
        parsed = pd.to_datetime(df_pos[col], errors="coerce")
        # only replace if parsing worked for a meaningful portion
        if parsed.notna().mean() > 0.2:
            df_pos[col] = parsed

text_cols = ["jobtitle_raw", "company", "location", "mapped_role", "job_category", "seniority"]
for c in text_cols:
    if c in df_pos.columns:
        df_pos[c] = df_pos[c].astype("string").str.strip()

print("\nNull counts (top 15):")
display(df_pos.isna().sum().sort_values(ascending=False).head(15))

# Keep df_pos in memory for the next steps (title normalization + SOC mapping)

Extracted to: ../data/sample_data_extracted
Top-level extracted files:
 - __MACOSX/._sample_data
 - __MACOSX/sample_data/._.DS_Store
 - __MACOSX/sample_data/._individual_data
 - __MACOSX/sample_data/._occupation_requirements
 - __MACOSX/sample_data/individual_data/._individual_position.csv
 - __MACOSX/sample_data/individual_data/._individual_user_education.csv
 - __MACOSX/sample_data/individual_data/._individual_user_skill.csv
 - __MACOSX/sample_data/occupation_requirements/._Alternate Titles.xlsx
 - __MACOSX/sample_data/occupation_requirements/._Education, Training, and Experience Categories.xlsx
 - __MACOSX/sample_data/occupation_requirements/._Education, Training, and Experience.xlsx
 - __MACOSX/sample_data/occupation_requirements/._Sample of Reported Titles.xlsx
 - __MACOSX/sample_data/occupation_requirements/._Scales Reference.xlsx
 - __MACOSX/sample_data/occupation_requirements/._Task Categories.xlsx
 - __MACOSX/sample_data/occupation_requirements/._Task Ratings.xlsx
 - sample_da

,user_id,position_id,company_raw,company_linkedin_url,company_cleaned,region,country,state,metro_area,startdate,...,jobtitle_raw,mapped_role,job_category,seniority,salary,rn,rcid,company_name,ultimate_parent_rcid,ultimate_parent_company_name
0,840332626,5611932128962179710,labor ready,linkedin.com/company/labor-ready,labor ready,Northern America,United States,California,long beach metropolitan area,2012-01-01,...,laborer,laborer,Engineer,1,56873.871123,1,306661,"TrueBlue, Inc.",306661,"TrueBlue, Inc."
1,841151205,-3931130892213672808,Illinois Institute of Technology,linkedin.com/company/illinois-tech,illinois institute of technology,Northern America,United States,Illinois,illinois nonmetropolitan area,2014-08-01,...,Teaching Assistant,teaching,Admin,1,50440.059792,1,838160,Illinois Institute of Technology,838160,Illinois Institute of Technology
2,841151205,8045923659602219163,J.P. Morgan,linkedin.com/company/jpmorgan,j p morgan,Northern America,United States,New York,new york nonmetropolitan area,2015-06-01,...,Software Engineering Intern,software engineering,Engineer,1,73077.956889,2,220821,"JPMorgan Investment Management, Inc.",543448,JPMorgan Chase & Co.
3,841151205,2701527518180934831,Goldman Sachs,linkedin.com/company/goldman-sachs,goldman sachs,Northern America,United States,New York,new york nonmetropolitan area,2016-06-01,...,Software Engineering Intern,software engineering,Engineer,1,87353.471630,3,20923715,"The Goldman Sachs Group, Inc.",20923715,"The Goldman Sachs Group, Inc."
4,841151205,-3315325678691742263,Goldman Sachs,linkedin.com/company/goldman-sachs,goldman sachs,Northern America,United States,New York,new york nonmetropolitan area,2017-02-01,...,"Analyst, Software Engineer",software engineer,Engineer,2,107378.051465,4,20923715,"The Goldman Sachs Group, Inc.",20923715,"The Goldman Sachs Group, Inc."
5,841151205,-5108328541936926815,Goldman Sachs,linkedin.com/company/goldman-sachs,goldman sachs,Northern America,United States,New York,new york nonmetropolitan area,2020-01-01,...,"Associate, Software Engineer",software engineering,Engineer,2,141995.306838,5,20923715,"The Goldman Sachs Group, Inc.",20923715,"The Goldman Sachs Group, Inc."
6,841151205,-6242556914544825950,Goldman Sachs,linkedin.com/company/goldman-sachs,goldman sachs,Northern America,United States,New York,new york nonmetropolitan area,2023-01-01,...,"Vice President, Software Engineer",software engineer,Engineer,2,218177.721485,6,20923715,"The Goldman Sachs Group, Inc.",20923715,"The Goldman Sachs Group, Inc."
7,841403523,-5933975230754183227,DPT Laboratories,linkedin.com/company/dpt-laboratories,dpt laboratories,Northern America,United States,New Jersey,new jersey nonmetropolitan area,2015-10-01,...,Change Control Administrator,document controller,Engineer,1,66276.859260,1,545690,DPT Laboratories Ltd.,949223,"Viatris, Inc."
8,841403523,1527371377900994966,"Renaissance Lakewood, LLC",linkedin.com/company/renaissance-lakewood,renaissance lakewood llc,Northern America,United States,New Jersey,new york city metropolitan area,2016-05-01,...,Quality System Specialist,quality auditor,Engineer,1,62788.484683,2,1277436,Renaissance Lakewood LLC,20947107,"Roundtable Healthcare Management, Inc."
9,841403523,3895107283803747500,"ARTEGRAFT, INC.",linkedin.com/company/artegraft-inc.,artegraft inc,Northern America,United States,North Carolina,raleigh metropolitan area,2017-09-01,...,Quality Compliance Associate,quality auditor,Engineer,2,59701.355808,3,465282,"Artegraft, Inc.",1052785,"LeMaitre Vascular, Inc."



Null counts (top 15):


enddate                         1004
ultimate_parent_company_name       2
company_name                       2
company_raw                        1
user_id                            0
company_cleaned                    0
company_linkedin_url               0
position_id                        0
region                             0
metro_area                         0
startdate                          0
state                              0
country                            0
mapped_role                        0
jobtitle_raw                       0
dtype: int64

In [2]:
import re
import unicodedata
import pandas as pd

# ----------------------------
# Step 2: Normalize job titles
# ----------------------------

# Common modifiers to remove (we'll keep the "core" title)
TITLE_STOPWORDS = {
    "intern", "internship",
    "junior", "jr", "jr.",
    "senior", "sr", "sr.",
    "lead", "principal",
    "associate", "asst", "assistant",
    "staff", "chief", "head",
    "i", "ii", "iii", "iv", "v",  # roman numerals and levels often used as suffixes
    "level", "l1", "l2", "l3", "l4", "l5",
    "contract", "contractor", "temp", "temporary",
    "part-time", "part time", "full-time", "full time",
}

# Tokens we usually want to keep if they appear (don’t remove these)
# (you can expand later)
KEEP_TOKENS = {"engineer", "developer", "scientist", "analyst", "manager", "director", "product", "data"}

def _ascii_fold(s: str) -> str:
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def normalize_title(title) -> str:
    if pd.isna(title):
        return ""
    s = str(title).strip().lower()
    s = _ascii_fold(s)

    # Replace separators with space
    s = re.sub(r"[/|,_\-]+", " ", s)

    # Remove content in parentheses/brackets
    s = re.sub(r"\([^)]*\)", " ", s)
    s = re.sub(r"\[[^\]]*\]", " ", s)

    # Remove non-alphanumeric except spaces
    s = re.sub(r"[^a-z0-9\s]", " ", s)

    # Collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()

    # Tokenize
    toks = s.split()

    # Remove stopwords unless they are important tokens (rare case)
    cleaned = []
    for t in toks:
        if t in TITLE_STOPWORDS and t not in KEEP_TOKENS:
            continue
        cleaned.append(t)

    # Remove trailing roman numerals / levels (common pattern like "engineer ii")
    while cleaned and cleaned[-1] in {"i","ii","iii","iv","v","1","2","3","4","5"}:
        cleaned.pop()

    return " ".join(cleaned).strip()

df_pos["jobtitle_norm"] = df_pos["jobtitle_raw"].apply(normalize_title)

# Show examples
cols_show = ["jobtitle_raw", "jobtitle_norm"]
display(df_pos[cols_show].dropna().head(100))

print("\nUnique raw titles:", df_pos["jobtitle_raw"].nunique(dropna=True))
print("Unique normalized titles:", df_pos["jobtitle_norm"].nunique(dropna=True))

# Quick sanity check: most common normalized titles
display(df_pos["jobtitle_norm"].value_counts().head(25))


,jobtitle_raw,jobtitle_norm
0,laborer,laborer
1,Teaching Assistant,teaching
2,Software Engineering Intern,software engineering
3,Software Engineering Intern,software engineering
4,"Analyst, Software Engineer",analyst software engineer
...,...,...
95,Technician,technician
96,Construction Worker,construction worker
97,Machine Operator,machine operator
98,Heavy Equipment Operator,heavy equipment operator



Unique raw titles: 2660
Unique normalized titles: 2410


jobtitle_norm
                                   51
software engineer                  46
manager                            33
consultant                         28
project manager                    21
research                           19
operations manager                 16
general manager                    16
sales                              16
teaching                           15
registered nurse                   14
customer service representative    14
professor                          13
analyst                            13
software developer                 13
accountant                         12
administrative                     11
student                            11
engineer                           10
cashier                            10
key account manager                10
human resources                     9
systems engineer                    9
data analyst                        8
business development manager        8
Name: count, dtype: int64

In [3]:
from pathlib import Path
import pandas as pd

OUT_ARTIFACTS = Path("../data/artifacts")
OUT_ARTIFACTS.mkdir(parents=True, exist_ok=True)

# 1) Always save a CSV (most robust)
csv_path = OUT_ARTIFACTS / "positions_step2.csv"
df_pos.to_csv(csv_path, index=False)
print("Saved CSV:", csv_path)

# 2) Try Parquet safely by coercing problematic dtypes
df_safe = df_pos.copy()

# Coerce all "object" / pandas string dtypes to plain python strings
for c in df_safe.columns:
    if pd.api.types.is_object_dtype(df_safe[c]) or pd.api.types.is_string_dtype(df_safe[c]):
        df_safe[c] = df_safe[c].astype("string").fillna(pd.NA)

# If you have list/dict columns later, stringify them here:
# df_safe["some_col"] = df_safe["some_col"].astype(str)

parquet_path = OUT_ARTIFACTS / "positions_step2.parquet"
try:
    df_safe.to_parquet(parquet_path, index=False, engine="pyarrow")
    print("Saved Parquet (pyarrow):", parquet_path)
except Exception as e:
    print("Parquet save failed, falling back to CSV-only. Error:\n", repr(e))


Saved CSV: ../data/artifacts/positions_step2.csv
Saved Parquet (pyarrow): ../data/artifacts/positions_step2.parquet


In [4]:
!pip install openpyxl
from pathlib import Path
import pandas as pd

BASE = Path("../data/sample_data_extracted/sample_data/occupation_requirements")

alt_path = BASE / "Alternate Titles.xlsx"
rep_path = BASE / "Sample of Reported Titles.xlsx"

df_alt = pd.read_excel(alt_path)
df_rep = pd.read_excel(rep_path)

print("Alternate Titles:", df_alt.shape)
print("Reported Titles:", df_rep.shape)

display(df_alt.head(5))
display(df_rep.head(5))

print("\nAlternate Titles columns:", list(df_alt.columns))
print("Reported Titles columns:", list(df_rep.columns))


Alternate Titles: (58348, 5)
Reported Titles: (9248, 4)


,O*NET-SOC Code,Title,Alternate Title,Short Title,Source(s)
0,11-1011.00,Chief Executives,Aeronautics Commission Director,NaN,08
1,11-1011.00,Chief Executives,Agricultural Services Director,NaN,08
2,11-1011.00,Chief Executives,Alcohol and Drug Abuse Assistance Program Admi...,NaN,08
3,11-1011.00,Chief Executives,Arts and Humanities Council Director,NaN,08
4,11-1011.00,Chief Executives,Bakery Manager,NaN,08


,O*NET-SOC Code,Title,Reported Job Title,Shown in My Next Move
0,11-1011.00,Chief Executives,Chief Diversity Officer (CDO),N
1,11-1011.00,Chief Executives,Chief Executive Officer (CEO),Y
2,11-1011.00,Chief Executives,Chief Financial Officer (CFO),Y
3,11-1011.00,Chief Executives,Chief Nursing Officer,N
4,11-1011.00,Chief Executives,Chief Operating Officer (COO),N



Alternate Titles columns: ['O*NET-SOC Code', 'Title', 'Alternate Title', 'Short Title', 'Source(s)']
Reported Titles columns: ['O*NET-SOC Code', 'Title', 'Reported Job Title', 'Shown in My Next Move']


In [5]:
import pandas as pd

# ----------------------------
# Step 3: Build O*NET title dictionary
# ----------------------------

def clean_soc(s):
    if pd.isna(s):
        return ""
    return str(s).strip()

def clean_title(s):
    if pd.isna(s):
        return ""
    return str(s).strip()

# 1) Alternate titles -> long format
alt_cols = ['O*NET-SOC Code', 'Title', 'Alternate Title', 'Short Title', 'Source(s)']
df_alt_ = df_alt[alt_cols].copy()

alt_rows = []

# canonical Title
alt_rows.append(
    df_alt_[['O*NET-SOC Code', 'Title']]
    .rename(columns={'O*NET-SOC Code':'soc_code', 'Title':'title'})
    .assign(source='canonical')
)

# alternate title
alt_rows.append(
    df_alt_[['O*NET-SOC Code', 'Alternate Title']]
    .rename(columns={'O*NET-SOC Code':'soc_code', 'Alternate Title':'title'})
    .assign(source='alternate')
)

# short title
alt_rows.append(
    df_alt_[['O*NET-SOC Code', 'Short Title']]
    .rename(columns={'O*NET-SOC Code':'soc_code', 'Short Title':'title'})
    .assign(source='short')
)

onet_titles_alt = pd.concat(alt_rows, ignore_index=True)
onet_titles_alt["soc_code"] = onet_titles_alt["soc_code"].apply(clean_soc)
onet_titles_alt["title"] = onet_titles_alt["title"].apply(clean_title)

# drop blanks
onet_titles_alt = onet_titles_alt[onet_titles_alt["soc_code"].ne("") & onet_titles_alt["title"].ne("")]

# 2) Reported titles -> long format
rep_cols = ['O*NET-SOC Code', 'Title', 'Reported Job Title', 'Shown in My Next Move']
df_rep_ = df_rep[rep_cols].copy()

rep_rows = []
# canonical Title
rep_rows.append(
    df_rep_[['O*NET-SOC Code', 'Title']]
    .rename(columns={'O*NET-SOC Code':'soc_code', 'Title':'title'})
    .assign(source='canonical_reported_sheet')
)
# reported job title
rep_rows.append(
    df_rep_[['O*NET-SOC Code', 'Reported Job Title']]
    .rename(columns={'O*NET-SOC Code':'soc_code', 'Reported Job Title':'title'})
    .assign(source='reported')
)

onet_titles_rep = pd.concat(rep_rows, ignore_index=True)
onet_titles_rep["soc_code"] = onet_titles_rep["soc_code"].apply(clean_soc)
onet_titles_rep["title"] = onet_titles_rep["title"].apply(clean_title)
onet_titles_rep = onet_titles_rep[onet_titles_rep["soc_code"].ne("") & onet_titles_rep["title"].ne("")]

# 3) Combine + normalize titles using your normalize_title() from Step 2
onet_titles = pd.concat([onet_titles_alt, onet_titles_rep], ignore_index=True)

# Remove exact duplicates
onet_titles = onet_titles.drop_duplicates(subset=["soc_code", "title", "source"]).reset_index(drop=True)

# Normalize
onet_titles["title_norm"] = onet_titles["title"].apply(normalize_title)

# Drop empty normalized titles (rare but possible)
onet_titles = onet_titles[onet_titles["title_norm"].ne("")].reset_index(drop=True)

print("O*NET titles table shape:", onet_titles.shape)
print("Unique SOC codes:", onet_titles["soc_code"].nunique())
print("Unique normalized titles:", onet_titles["title_norm"].nunique())

display(onet_titles.sample(10, random_state=0))
display(onet_titles["source"].value_counts())


O*NET titles table shape: (71226, 4)
Unique SOC codes: 1109
Unique normalized titles: 46333


,soc_code,title,source,title_norm
23798,41-2022.00,Parts Salesman,alternate,parts salesman
29806,45-4011.00,Christmas Tree Farm Worker,alternate,christmas tree farm worker
17935,29-1125.01,Art Psychotherapist,alternate,art psychotherapist
46751,51-9021.00,Platen Grinder,alternate,platen grinder
2502,11-9031.00,Center Administrator,alternate,center administrator
62977,13-2099.02,Risk Management Director,reported,risk management director
50646,51-9192.00,Degreaser,alternate,degreaser
52639,51-9199.00,Core Winder Machine Operator,alternate,core winder machine operator
43747,51-6062.00,Welt Cutter,alternate,welt cutter
61560,37-2011.00,"Janitors and Cleaners, Except Maids and Housek...",canonical_reported_sheet,janitors and cleaners except maids and houseke...


source
alternate                   58338
reported                     9244
short                        1567
canonical                    1109
canonical_reported_sheet      968
Name: count, dtype: int64

In [6]:
from pathlib import Path

OUT_ARTIFACTS = Path("../data/artifacts")
OUT_ARTIFACTS.mkdir(parents=True, exist_ok=True)

onet_csv = OUT_ARTIFACTS / "onet_titles_step3.csv"
onet_titles.to_csv(onet_csv, index=False)
print("Saved:", onet_csv)

# Optional parquet (may fail depending on your env, CSV is enough)
try:
    onet_parquet = OUT_ARTIFACTS / "onet_titles_step3.parquet"
    onet_titles.to_parquet(onet_parquet, index=False, engine="pyarrow")
    print("Saved:", onet_parquet)
except Exception as e:
    print("Parquet save skipped (CSV is fine). Error:", repr(e))


Saved: ../data/artifacts/onet_titles_step3.csv
Saved: ../data/artifacts/onet_titles_step3.parquet


In [7]:
import pandas as pd

# ----------------------------
# Step 4A: Exact match jobtitle_norm -> SOC
# ----------------------------

# Reduce O*NET titles to unique normalized title -> SOC candidates
onet_norm_to_soc = (
    onet_titles[["title_norm", "soc_code"]]
    .drop_duplicates()
    .groupby("title_norm")["soc_code"]
    .apply(list)
    .reset_index()
    .rename(columns={"soc_code": "soc_candidates"})
)

# Merge exact matches
df_map_exact = df_pos[["position_id", "jobtitle_raw", "jobtitle_norm"]].merge(
    onet_norm_to_soc, how="left", left_on="jobtitle_norm", right_on="title_norm"
)

df_map_exact["exact_match"] = df_map_exact["soc_candidates"].notna()

exact_rate = df_map_exact["exact_match"].mean()
print(f"Exact match rate (positions): {exact_rate:.3%}")

# How many distinct jobtitle_norm matched exactly?
unique_titles = df_map_exact[["jobtitle_norm", "exact_match"]].drop_duplicates()
print("Exact match rate (unique normalized titles):",
      f"{unique_titles['exact_match'].mean():.3%}")

# If multiple SOC candidates exist for a title_norm, keep them for now.
# We'll select top1 later (or keep multi-mapping and score downstream).
display(df_map_exact[df_map_exact["exact_match"]].head(10))


Exact match rate (positions): 40.078%
Exact match rate (unique normalized titles): 25.892%


,position_id,jobtitle_raw,jobtitle_norm,title_norm,soc_candidates,exact_match
0,5611932128962179710,laborer,laborer,laborer,"[37-3011.00, 37-3013.00, 45-2099.00, 45-4029.0...",True
1,-3931130892213672808,Teaching Assistant,teaching,teaching,"[25-1191.00, 25-9041.00]",True
5,-5108328541936926815,"Associate, Software Engineer",software engineer,software engineer,"[15-1132.00, 15-1133.00]",True
11,1934320596464703875,Regulatory Compliance Manager,regulatory compliance manager,regulatory compliance manager,[11-9199.02],True
12,8027492298332992536,Senior Compliance Specialist,compliance specialist,compliance specialist,[13-2061.00],True
13,-7619093065810600201,Senior Training Specialist,training specialist,training specialist,"[13-1151.00, 19-3032.00]",True
14,-1576140437181324427,Operations Manager,operations manager,operations manager,"[11-1021.00, 11-3051.02, 11-3051.03, 11-3051.0...",True
15,6039986263365326192,Retail,retail,retail,[41-2031.00],True
16,687348313092154833,Cafeteria Manager,cafeteria manager,cafeteria manager,[35-1012.00],True
17,-8340296190348302004,Engineer,engineer,engineer,"[11-9041.00, 17-2021.00, 17-2031.00, 17-2041.0...",True


In [8]:
!pip -q install rapidfuzz

from rapidfuzz import process, fuzz

# ----------------------------
# Step 4B: Fuzzy match for non-exact titles
# ----------------------------

# Candidate set: all unique O*NET normalized titles
onet_title_norms = onet_norm_to_soc["title_norm"].tolist()

# Get unique normalized titles in our positions that are NOT exact matched
unmatched_titles = (
    df_map_exact.loc[~df_map_exact["exact_match"], "jobtitle_norm"]
    .dropna()
    .unique()
    .tolist()
)

print("Unmatched unique normalized titles:", len(unmatched_titles))

def best_fuzzy_match(q: str):
    """
    Returns (best_title_norm, score, soc_candidates) or (None, 0, None)
    """
    if not q or pd.isna(q):
        return (None, 0, None)
    # token_set_ratio is robust to word order
    hit = process.extractOne(q, onet_title_norms, scorer=fuzz.token_set_ratio)
    if hit is None:
        return (None, 0, None)
    best_title, score, _ = hit
    socs = onet_norm_to_soc.loc[onet_norm_to_soc["title_norm"] == best_title, "soc_candidates"].iloc[0]
    return (best_title, score, socs)

# Run fuzzy matching for unmatched titles
fuzzy_rows = []
for t in unmatched_titles:
    best_title, score, socs = best_fuzzy_match(t)
    fuzzy_rows.append((t, best_title, score, socs))

df_fuzzy = pd.DataFrame(fuzzy_rows, columns=["jobtitle_norm", "best_title_norm", "fuzzy_score", "soc_candidates"])

# Choose a threshold; 90 is conservative, 85 is usually ok.
THRESH = 90
df_fuzzy["fuzzy_match"] = df_fuzzy["fuzzy_score"] >= THRESH

print("Fuzzy match rate among unmatched titles:",
      f"{df_fuzzy['fuzzy_match'].mean():.3%} at threshold {THRESH}")

display(df_fuzzy.sort_values("fuzzy_score", ascending=False).head(20))
display(df_fuzzy.sort_values("fuzzy_score", ascending=True).head(20))


Unmatched unique normalized titles: 1786
Fuzzy match rate among unmatched titles: 79.171% at threshold 90


,jobtitle_norm,best_title_norm,fuzzy_score,soc_candidates,fuzzy_match
1785,municipal bond trading,trading,100.0,[43-4011.00],True
0,software engineering,applications software engineering information ...,100.0,[15-1132.00],True
1,analyst software engineer,analyst,100.0,"[13-1023.00, 13-1051.00, 13-2031.00, 13-2041.0...",True
2,vice president software engineer,engineer,100.0,"[11-9041.00, 17-2021.00, 17-2031.00, 17-2041.0...",True
3,change control administrator,administrator,100.0,"[11-3011.00, 11-3061.00, 11-9031.00, 13-1023.0...",True
4,quality system specialist,quality,100.0,[51-9061.00],True
5,quality compliance,quality,100.0,[51-9061.00],True
6,quality compliance specialist,compliance specialist,100.0,[13-2061.00],True
7,department engineering,department,100.0,"[11-9071.00, 51-1011.00]",True
8,blood collections,collections,100.0,[43-3011.00],True


,jobtitle_norm,best_title_norm,fuzzy_score,soc_candidates,fuzzy_match
36,,NaN,0.000000,None,False
390,immobilienverkauf und objektbegutachtung,mobile radio technician,47.619048,[49-2021.01],False
1382,fertigungsassistent hochdruck gasentladungslam...,finish production manager,51.764706,[51-1011.00],False
240,stagiaire planification financiere et partenar...,hydro mechanic,52.631579,"[49-3042.00, 51-8099.04]",False
179,logistiek en verkoop medewerker,logistics team member,53.846154,[13-1081.00],False
1001,analista gerencia desarrollo inmobiliario,reconciliation analyst,53.968254,[43-3031.00],False
979,buroleiterin fritz guntzler mdb,boiler fitter,54.545455,[47-2011.00],False
977,fachreferentin maik beermann mdb,engineering model maker,54.545455,[51-4061.00],False
1298,gerente desenvolvimento agronomico,carbonation equipment tender,54.838710,"[51-9012.00, 51-9023.00]",False
1780,consultora de inovacao e negocios em sap,medical social consultant,55.384615,[21-1022.00],False


In [15]:
# ----------------------------
# Step 4C: Attach SOC mapping to df_pos
# ----------------------------

# Build a title-level mapping table (jobtitle_norm -> mapping result)
df_exact_title_map = (
    df_map_exact[["jobtitle_norm", "soc_candidates"]]
    .dropna()
    .drop_duplicates(subset=["jobtitle_norm"])
    .assign(match_type="exact", match_score=100.0)
)

df_fuzzy_title_map = (
    df_fuzzy[df_fuzzy["fuzzy_match"]]
    .copy()
    .rename(columns={"soc_candidates": "soc_candidates_fuzzy"})
)

df_fuzzy_title_map = df_fuzzy_title_map[["jobtitle_norm", "soc_candidates_fuzzy", "fuzzy_score"]]
df_fuzzy_title_map = df_fuzzy_title_map.rename(columns={
    "soc_candidates_fuzzy": "soc_candidates",
    "fuzzy_score": "match_score"
}).assign(match_type="fuzzy")

# Combine: exact first, then fuzzy for titles not in exact
title_map = pd.concat([df_exact_title_map, df_fuzzy_title_map], ignore_index=True)

# If a title appears in both (shouldn't, but safe), keep exact
title_map = title_map.sort_values(["jobtitle_norm", "match_type"]).drop_duplicates("jobtitle_norm", keep="first")

print("Title map size:", title_map.shape)
print("Mapped unique normalized titles:", title_map["jobtitle_norm"].nunique())

# Merge back to positions
df_pos = df_pos.merge(title_map, how="left", on="jobtitle_norm")

# Create a top-1 SOC code (for now: first candidate)
df_pos["soc_code_top1"] = df_pos["soc_candidates"].apply(lambda x: x[0] if isinstance(x, list) and len(x) else pd.NA)

print("Mapped positions:", df_pos["soc_code_top1"].notna().mean())
display(df_pos[["jobtitle_raw", "jobtitle_norm", "match_type", "match_score", "soc_code_top1"]].head(20))


Title map size: (2038, 4)
Mapped unique normalized titles: 2038
Mapped positions: 0.8643156007172743


,jobtitle_raw,jobtitle_norm,match_type,match_score,soc_code_top1
0,laborer,laborer,exact,100.0,37-3011.00
1,Teaching Assistant,teaching,exact,100.0,25-1191.00
2,Software Engineering Intern,software engineering,fuzzy,100.0,15-1132.00
3,Software Engineering Intern,software engineering,fuzzy,100.0,15-1132.00
4,"Analyst, Software Engineer",analyst software engineer,fuzzy,100.0,13-1023.00
5,"Associate, Software Engineer",software engineer,exact,100.0,15-1132.00
6,"Vice President, Software Engineer",vice president software engineer,fuzzy,100.0,11-9041.00
7,Change Control Administrator,change control administrator,fuzzy,100.0,11-3011.00
8,Quality System Specialist,quality system specialist,fuzzy,100.0,51-9061.00
9,Quality Compliance Associate,quality compliance,fuzzy,100.0,51-9061.00


In [16]:
from pathlib import Path

OUT_ARTIFACTS = Path("../data/artifacts")
OUT_ARTIFACTS.mkdir(parents=True, exist_ok=True)

mapped_pos_csv = OUT_ARTIFACTS / "positions_step4_with_soc.csv"
df_pos.to_csv(mapped_pos_csv, index=False)
print("Saved:", mapped_pos_csv)

title_map_csv = OUT_ARTIFACTS / "jobtitle_to_soc_step4.csv"
title_map.to_csv(title_map_csv, index=False)
print("Saved:", title_map_csv)


Saved: ../data/artifacts/positions_step4_with_soc.csv
Saved: ../data/artifacts/jobtitle_to_soc_step4.csv


In [17]:
import re
import numpy as np

# ----------------------------
# Step 4E: Heuristics to flag non-English titles + safer fuzzy usage
# ----------------------------

# Very lightweight "English-ness" heuristic:
# - if it contains many non-ascii letters OR common German/French/Spanish markers, flag it
NON_EN_MARKERS = set(list("äöüßéèêáàâíìîóòôúùûñçøåœæ"))

def looks_non_english(s: str) -> bool:
    if s is None or pd.isna(s):
        return False
    s = str(s).strip().lower()
    if not s:
        return False

    # Non-ascii letters present?
    if any(ch in NON_EN_MARKERS for ch in s):
        return True

    # Contains many tokens that are unlikely in English (very rough)
    # You can extend this list as you see new languages in the data.
    common_non_en_words = {
        "und","oder","der","die","das","für","mit","auf","bei","im","zum","zur",
        "ingenieur","entwickler","verkauf","objekt","immobilien",
        "gestion","directeur","ingeniero","ventas","técnico","tecnico"
    }
    toks = re.findall(r"[a-zA-ZÀ-ÿ]+", s)
    hit = sum(t in common_non_en_words for t in toks)
    if hit >= 1:
        return True

    # If more than ~30% of chars are non-ascii, probably non-English
    non_ascii = sum(ord(c) > 127 for c in s)
    if non_ascii / max(1, len(s)) > 0.3:
        return True

    return False

df_pos["title_non_english_flag"] = df_pos["jobtitle_raw"].apply(looks_non_english)

print("Non-English flagged positions:", df_pos["title_non_english_flag"].mean())
display(df_pos.loc[df_pos["title_non_english_flag"], ["jobtitle_raw","jobtitle_norm","match_type","match_score","soc_code_top1"]].head(25))

# Tighten fuzzy threshold for non-English titles
# Rule: if non-English and fuzzy match, require higher score
THRESH_EN = 90
THRESH_NON_EN = 97  # stricter, because fuzzy on foreign strings is unreliable

def should_accept_mapping(row):
    if pd.isna(row.get("soc_code_top1")):
        return False
    if row.get("match_type") == "exact":
        return True
    if row.get("match_type") == "fuzzy":
        score = row.get("match_score")
        if pd.isna(score):
            return False
        if row.get("title_non_english_flag"):
            return score >= THRESH_NON_EN
        return score >= THRESH_EN
    return False

df_pos["soc_accept"] = df_pos.apply(should_accept_mapping, axis=1)
df_pos["soc_code_final"] = np.where(df_pos["soc_accept"], df_pos["soc_code_top1"], pd.NA)

print("Accepted mappings (positions):", pd.notna(df_pos["soc_code_final"]).mean())

# Show examples of rejected fuzzy matches
display(
    df_pos.loc[
        (df_pos["match_type"] == "fuzzy") & (~df_pos["soc_accept"]),
        ["jobtitle_raw","jobtitle_norm","match_score","soc_code_top1","title_non_english_flag"]
    ].head(25)
)


Non-English flagged positions: 0.04662283323371189


,jobtitle_raw,jobtitle_norm,match_type,match_score,soc_code_top1
32,Stagiaire en développement Web,stagiaire en developpement web,NaN,NaN,NaN
33,Développeur,developpeur,fuzzy,90.000000,11-9021.00
38,Étudiant stagiaire,etudiant stagiaire,NaN,NaN,NaN
40,Chargée de la commande publique,chargee de la commande publique,NaN,NaN,NaN
48,Programador de Produção,programador de producao,NaN,NaN,NaN
86,Comptable général,comptable general,fuzzy,100.000000,11-9021.00
92,Iniciação Científica,iniciacao cientifica,NaN,NaN,NaN
99,Técnico de torre,tecnico de torre,NaN,NaN,NaN
100,Técnico de telecomunicaciones,tecnico de telecomunicaciones,NaN,NaN,NaN
234,mecanico de manutenção,mecanico de manutencao,NaN,NaN,NaN


Accepted mappings (positions): 0.8637178720860729


,jobtitle_raw,jobtitle_norm,match_score,soc_code_top1,title_non_english_flag
33,Développeur,developpeur,90.000000,11-9021.00,True
290,Développeur web,developpeur web,92.857143,15-1134.00,True


In [18]:
# ----------------------------
# Step 4F (Optional): Fallback mapping via mapped_role/job_category
# ----------------------------

# Build a role->SOC distribution from HIGH-confidence mappings
high_conf = df_pos[(df_pos["soc_accept"]) & (df_pos["match_type"] == "exact")].copy()

if "mapped_role" in df_pos.columns:
    role_to_soc = (
        high_conf.groupby("mapped_role")["soc_code_top1"]
        .agg(lambda s: s.value_counts().head(1).index[0] if len(s) else pd.NA)
        .dropna()
        .to_dict()
    )

    # Apply fallback only where soc_code_final is missing
    missing = df_pos["soc_code_final"].isna()
    df_pos.loc[missing, "soc_code_fallback_role"] = df_pos.loc[missing, "mapped_role"].map(role_to_soc)

    # If we got something, fill soc_code_final
    df_pos["soc_code_final"] = df_pos["soc_code_final"].fillna(df_pos["soc_code_fallback_role"])

    print("After role fallback, mapped positions:",
          pd.notna(df_pos["soc_code_final"]).mean())

    display(df_pos.loc[missing & df_pos["soc_code_fallback_role"].notna(),
                       ["jobtitle_raw","mapped_role","soc_code_fallback_role"]].head(20))
else:
    print("No mapped_role column found; skipping fallback.")


After role fallback, mapped positions: 0.9509862522414824


,jobtitle_raw,mapped_role,soc_code_fallback_role
32,Stagiaire en développement Web,web developer,15-1134.00
33,Développeur,web developer,15-1134.00
38,Étudiant stagiaire,student,25-1191.00
47,Programador,programmer,15-1132.00
66,Monitoring And Evaluation Assistant,project officer,11-9021.00
73,Intern,student worker,43-4071.00
85,Collaborateur comptable,accounting,13-2011.00
92,Iniciação Científica,research,11-9041.01
94,Analista de Risco Socioambiental Jr,research,11-9041.01
109,Directora de Controlling Corporativo,finance controller,11-1021.00


In [19]:
from pathlib import Path
import pandas as pd

REQ_BASE = Path("../data/sample_data_extracted/sample_data/occupation_requirements")

task_ratings_path = REQ_BASE / "Task Ratings.xlsx"
task_categories_path = REQ_BASE / "Task Categories.xlsx"
scales_ref_path = REQ_BASE / "Scales Reference.xlsx"

df_tasks = pd.read_excel(task_ratings_path)
df_task_cat = pd.read_excel(task_categories_path)
df_scales = pd.read_excel(scales_ref_path)

print("Task Ratings:", df_tasks.shape)
print("Task Categories:", df_task_cat.shape)
print("Scales Reference:", df_scales.shape)

print("\nTask Ratings columns:", list(df_tasks.columns))
print("\nScales Reference columns:", list(df_scales.columns))

display(df_tasks.head(5))
display(df_scales.head(10))


Task Ratings: (175482, 15)
Task Categories: (7, 4)
Scales Reference: (29, 4)

Task Ratings columns: ['O*NET-SOC Code', 'Title', 'Task ID', 'Task', 'Scale ID', 'Scale Name', 'Category', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound', 'Recommend Suppress', 'Date', 'Domain Source']

Scales Reference columns: ['Scale ID', 'Scale Name', 'Minimum', 'Maximum']


,O*NET-SOC Code,Title,Task ID,Task,Scale ID,Scale Name,Category,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Date,Domain Source
0,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),1.0,4.34,79,2.48,1.36,12.96,N,07/2014,Incumbent
1,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),2.0,9.16,79,3.86,3.86,20.24,N,07/2014,Incumbent
2,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),3.0,11.04,79,3.44,5.82,19.95,N,07/2014,Incumbent
3,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),4.0,16.19,79,4.37,9.24,26.83,N,07/2014,Incumbent
4,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),5.0,46.67,79,6.03,35.07,58.64,N,07/2014,Incumbent


,Scale ID,Scale Name,Minimum,Maximum
0,AO,Automation,1,5
1,CF,Frequency,1,5
2,CN,Amount of Contact,1,5
3,CT,Context,1,3
4,CTP,Context (Categories 1-3),0,100
5,CX,Context,1,5
6,CXP,Context (Categories 1-5),0,100
7,EX,Extent,1,7
8,FM,Amount of Freedom,1,5
9,FT,Frequency of Task (Categories 1-7),0,100


In [20]:
from pathlib import Path
import pandas as pd

# ----------------------------
# Step 5A: Load O*NET task + scale tables
# ----------------------------

REQ_BASE = Path("../data/sample_data_extracted/sample_data/occupation_requirements")

task_ratings_path = REQ_BASE / "Task Ratings.xlsx"
task_categories_path = REQ_BASE / "Task Categories.xlsx"
scales_ref_path = REQ_BASE / "Scales Reference.xlsx"

df_tasks = pd.read_excel(task_ratings_path)
df_task_cat = pd.read_excel(task_categories_path)
df_scales = pd.read_excel(scales_ref_path)

print("Task Ratings:", df_tasks.shape)
print("Task Categories:", df_task_cat.shape)
print("Scales Reference:", df_scales.shape)

print("\nTask Ratings columns:")
print(list(df_tasks.columns))

print("\nScales Reference columns:")
print(list(df_scales.columns))

print("\nScales Reference preview:")
display(df_scales.head(25))

print("\nTask Ratings preview:")
display(df_tasks.head(10))


Task Ratings: (175482, 15)
Task Categories: (7, 4)
Scales Reference: (29, 4)

Task Ratings columns:
['O*NET-SOC Code', 'Title', 'Task ID', 'Task', 'Scale ID', 'Scale Name', 'Category', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound', 'Recommend Suppress', 'Date', 'Domain Source']

Scales Reference columns:
['Scale ID', 'Scale Name', 'Minimum', 'Maximum']

Scales Reference preview:


,Scale ID,Scale Name,Minimum,Maximum
0,AO,Automation,1,5
1,CF,Frequency,1,5
2,CN,Amount of Contact,1,5
3,CT,Context,1,3
4,CTP,Context (Categories 1-3),0,100
5,CX,Context,1,5
6,CXP,Context (Categories 1-5),0,100
7,EX,Extent,1,7
8,FM,Amount of Freedom,1,5
9,FT,Frequency of Task (Categories 1-7),0,100



Task Ratings preview:


,O*NET-SOC Code,Title,Task ID,Task,Scale ID,Scale Name,Category,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Date,Domain Source
0,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),1.0,4.34,79,2.48,1.36,12.96,N,07/2014,Incumbent
1,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),2.0,9.16,79,3.86,3.86,20.24,N,07/2014,Incumbent
2,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),3.0,11.04,79,3.44,5.82,19.95,N,07/2014,Incumbent
3,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),4.0,16.19,79,4.37,9.24,26.83,N,07/2014,Incumbent
4,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),5.0,46.67,79,6.03,35.07,58.64,N,07/2014,Incumbent
5,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),6.0,7.33,79,3.40,2.83,17.65,N,07/2014,Incumbent
6,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,FT,Frequency of Task (Categories 1-7),7.0,5.26,79,4.30,0.99,23.59,N,07/2014,Incumbent
7,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,IM,Importance,NaN,4.54,78,0.08,4.38,4.71,N,07/2014,Incumbent
8,11-1011.00,Chief Executives,8823,Direct or coordinate an organization's financi...,RT,Relevance of Task,NaN,94.19,87,3.35,82.75,98.21,N,07/2014,Incumbent
9,11-1011.00,Chief Executives,8831,Appoint department heads or managers and assig...,FT,Frequency of Task (Categories 1-7),1.0,12.96,83,3.77,7.12,22.44,N,07/2014,Incumbent


In [21]:
# ----------------------------
# Step 5B: Build SOC × Task importance weights
# ----------------------------

# 1) Keep only Importance scale
df_task_imp = df_tasks[df_tasks["Scale ID"] == "IM"].copy()

print("Rows with IM scale:", df_task_imp.shape)

# Basic columns we need
df_task_imp = df_task_imp[[
    "O*NET-SOC Code",
    "Task ID",
    "Task",
    "Data Value"
]].rename(columns={
    "O*NET-SOC Code": "soc_code",
    "Data Value": "importance"
})

# Remove bad rows
df_task_imp = df_task_imp.dropna(subset=["soc_code", "Task ID", "importance"])

# Normalize importance to 0–1 (easier to interpret later)
df_task_imp["importance_norm"] = (df_task_imp["importance"] - 1) / (5 - 1)

print("Unique SOC:", df_task_imp["soc_code"].nunique())
print("Unique tasks:", df_task_imp["Task ID"].nunique())

display(df_task_imp.head(10))


Rows with IM scale: (19498, 15)
Unique SOC: 967
Unique tasks: 19498


,soc_code,Task ID,Task,importance,importance_norm
7,11-1011.00,8823,Direct or coordinate an organization's financi...,4.54,0.8850
16,11-1011.00,8831,Appoint department heads or managers and assig...,4.48,0.8700
25,11-1011.00,8825,Analyze operations to evaluate performance of ...,4.40,0.8500
34,11-1011.00,8826,"Direct, plan, or implement policies, objective...",4.39,0.8475
43,11-1011.00,8827,"Prepare budgets for approval, including those ...",4.17,0.7925
52,11-1011.00,8824,"Confer with board members, organization offici...",4.15,0.7875
61,11-1011.00,8836,Implement corrective action plans to solve org...,4.12,0.7800
70,11-1011.00,8832,"Direct human resources activities, including t...",4.02,0.7550
79,11-1011.00,8835,Establish departmental responsibilities and co...,3.96,0.7400
88,11-1011.00,8833,"Preside over or serve on boards of directors, ...",3.96,0.7400


In [22]:
# ----------------------------
# Step 5C: Map tasks to categories
# ----------------------------

print("Task category columns:", list(df_task_cat.columns))
display(df_task_cat)


Task category columns: ['Scale ID', 'Scale Name', 'Category', 'Category Description']


,Scale ID,Scale Name,Category,Category Description
0,FT,Frequency of Task (Categories 1-7),1,Yearly or less
1,FT,Frequency of Task (Categories 1-7),2,More than yearly
2,FT,Frequency of Task (Categories 1-7),3,More than monthly
3,FT,Frequency of Task (Categories 1-7),4,More than weekly
4,FT,Frequency of Task (Categories 1-7),5,Daily
5,FT,Frequency of Task (Categories 1-7),6,Several times daily
6,FT,Frequency of Task (Categories 1-7),7,Hourly or more


In [23]:
# ----------------------------
# Step 5D: Build "top tasks per SOC" requirement profile
# ----------------------------

TOP_K = 25  # start small; we can raise later

soc_top_tasks = (
    df_task_imp.sort_values(["soc_code", "importance"], ascending=[True, False])
    .groupby("soc_code")
    .head(TOP_K)
    .reset_index(drop=True)
)

print("Top-task rows:", soc_top_tasks.shape)
print("SOC coverage:", soc_top_tasks["soc_code"].nunique())
display(soc_top_tasks.head(20))

# Quick example: show top tasks for one SOC
example_soc = soc_top_tasks["soc_code"].iloc[0]
print("\nExample SOC:", example_soc)
display(soc_top_tasks[soc_top_tasks["soc_code"] == example_soc][["Task ID","Task","importance"]].head(10))


Top-task rows: (18617, 5)
SOC coverage: 967


,soc_code,Task ID,Task,importance,importance_norm
0,11-1011.00,8823,Direct or coordinate an organization's financi...,4.54,0.8850
1,11-1011.00,8831,Appoint department heads or managers and assig...,4.48,0.8700
2,11-1011.00,8825,Analyze operations to evaluate performance of ...,4.40,0.8500
3,11-1011.00,8826,"Direct, plan, or implement policies, objective...",4.39,0.8475
4,11-1011.00,8828,Direct or coordinate activities of businesses ...,4.25,0.8125
5,11-1011.00,8827,"Prepare budgets for approval, including those ...",4.17,0.7925
6,11-1011.00,8824,"Confer with board members, organization offici...",4.15,0.7875
7,11-1011.00,8836,Implement corrective action plans to solve org...,4.12,0.7800
8,11-1011.00,8832,"Direct human resources activities, including t...",4.02,0.7550
9,11-1011.00,8835,Establish departmental responsibilities and co...,3.96,0.7400



Example SOC: 11-1011.00


,Task ID,Task,importance
0,8823,Direct or coordinate an organization's financi...,4.54
1,8831,Appoint department heads or managers and assig...,4.48
2,8825,Analyze operations to evaluate performance of ...,4.40
3,8826,"Direct, plan, or implement policies, objective...",4.39
4,8828,Direct or coordinate activities of businesses ...,4.25
5,8827,"Prepare budgets for approval, including those ...",4.17
6,8824,"Confer with board members, organization offici...",4.15
7,8836,Implement corrective action plans to solve org...,4.12
8,8832,"Direct human resources activities, including t...",4.02
9,8835,Establish departmental responsibilities and co...,3.96


In [24]:
# ----------------------------
# Step 5E: Build a global task vocabulary
# ----------------------------

# Count how often tasks appear in SOC top-k lists
task_vocab = (
    soc_top_tasks.groupby(["Task ID", "Task"])
    .size()
    .reset_index(name="soc_count")
    .sort_values("soc_count", ascending=False)
)

print("Total unique tasks in SOC top-k:", task_vocab.shape[0])
display(task_vocab.head(30))

# Keep the most common tasks to define a consistent vector space
VOCAB_SIZE = 2000  # safe; adjust later
task_vocab_top = task_vocab.head(VOCAB_SIZE).copy()

# Create a lookup: Task ID -> column index
task_to_col = {tid: i for i, tid in enumerate(task_vocab_top["Task ID"].tolist())}

print("Vocab size:", len(task_to_col))


Total unique tasks in SOC top-k: 18617


,Task ID,Task,soc_count
0,1,Resolve customer complaints regarding sales an...,1
1,2,Monitor customer preferences to determine focu...,1
2,3,Direct and coordinate activities involving sal...,1
3,4,Determine price schedules and discount rates.,1
4,5,Review operational records and reports to proj...,1
5,6,"Direct, coordinate, and review activities in s...",1
6,7,Confer or consult with department heads to pla...,1
7,8,Advise dealers and distributors on policies an...,1
8,9,Prepare budgets and approve budget expenditures.,1
9,10,Represent company at trade association meeting...,1


Vocab size: 2000


In [25]:
# ----------------------------
# Step 5F: SOC × Task requirements (restricted to vocab)
# ----------------------------

soc_task_req = soc_top_tasks[soc_top_tasks["Task ID"].isin(task_to_col.keys())].copy()

# Keep only what we need
soc_task_req = soc_task_req[["soc_code", "Task ID", "importance_norm", "importance", "Task"]]

print("SOC-task requirement rows:", soc_task_req.shape)
print("SOC coverage:", soc_task_req["soc_code"].nunique())
print("Task coverage:", soc_task_req["Task ID"].nunique())

display(soc_task_req.head(20))


SOC-task requirement rows: (2000, 5)
SOC coverage: 130
Task coverage: 2000


,soc_code,Task ID,importance_norm,importance,Task
44,11-1021.00,933,0.7600,4.04,Direct and coordinate activities of businesses...
51,11-1021.00,944,0.7050,3.82,"Perform sales floor work, such as greeting or ..."
55,11-1021.00,945,0.5850,3.34,Develop or implement product-marketing strateg...
56,11-1021.00,948,0.5700,3.28,Recommend locations for new facilities or over...
58,11-1021.00,947,0.5600,3.24,Direct non-merchandising departments of busine...
59,11-1021.00,949,0.4925,2.97,Plan store layouts or design displays.
85,11-2021.00,951,0.8250,4.30,"Identify, develop, or evaluate marketing strat..."
87,11-2021.00,952,0.7475,3.99,Evaluate the financial aspects of product deve...
88,11-2021.00,950,0.7175,3.87,"Develop pricing strategies, balancing firm obj..."
89,11-2021.00,957,0.6750,3.70,Compile lists describing product or service of...


In [26]:
from pathlib import Path

OUT_ARTIFACTS = Path("../data/artifacts")
OUT_ARTIFACTS.mkdir(parents=True, exist_ok=True)

soc_task_req_csv = OUT_ARTIFACTS / "soc_task_requirements_step5.csv"
soc_task_req.to_csv(soc_task_req_csv, index=False)
print("Saved:", soc_task_req_csv)


Saved: ../data/artifacts/soc_task_requirements_step5.csv


In [27]:
from pathlib import Path
import pandas as pd

SKILL_PATH = Path("../data/sample_data_extracted/sample_data/individual_data/individual_user_skill.csv")
df_skill = pd.read_csv(SKILL_PATH)

print("Skills shape:", df_skill.shape)
print("Skills columns:", list(df_skill.columns))
display(df_skill.head(10))

print("\nUnique users in skills:", df_skill["user_id"].nunique())
print("Unique skill_mapped:", df_skill["skill_mapped"].nunique())
print("Unique skill_k75:", df_skill["skill_k75"].nunique())


Skills shape: (9896, 4)
Skills columns: ['user_id', 'skill_raw', 'skill_mapped', 'skill_k75']


,user_id,skill_raw,skill_mapped,skill_k75
0,571813982,project management,project management,project management / project planning / constr...
1,212616767,visual merchandising,visual merchandising,retail / merchandising / fashion
2,147751294,visual studio,visual studio,c# / .net / asp.net
3,71284739,vendor management,vendor management,process improvement / program management / cro...
4,43876414,inventory management,inventory management,operations management / inventory management /...
5,493321187,capital markets,capital markets,investment banking / equities / capital markets
6,285497625,matlab,matlab,c++ / c / matlab
7,301214847,powerpoint,powerpoint,microsoft office / microsoft excel / microsoft...
8,51960251,security,security,process improvement / program management / cro...
9,172662222,marketing strategy,marketing strategy,marketing / marketing strategy / customer rela...



Unique users in skills: 339
Unique skill_mapped: 1725
Unique skill_k75: 75


In [28]:
import pandas as pd

# ----------------------------
# Step 6B: Build user skill "documents"
# ----------------------------

# Clean / normalize skill text (simple)
def norm_skill(s):
    if pd.isna(s):
        return ""
    return str(s).strip().lower()

df_skill["skill_text"] = df_skill["skill_mapped"].fillna(df_skill["skill_raw"]).apply(norm_skill)

# Group skills by user into a single document (space-separated)
user_skill_docs = (
    df_skill.groupby("user_id")["skill_text"]
    .apply(lambda s: " ".join([x for x in s.tolist() if x]))
    .reset_index()
    .rename(columns={"skill_text": "skill_doc"})
)

print("Users with skill docs:", user_skill_docs.shape[0])
display(user_skill_docs.head(10))


Users with skill docs: 339


,user_id,skill_doc
0,1350263,retail new store development visual merchandis...
1,2802690,microsoft office public speaking nonprofits so...
2,5776072,web services software project management quali...
3,9427471,variance analysis cost accounting business str...
4,10787733,public relations proofreading public speaking ...
5,15096978,policy strategic communications healthcare res...
6,16620582,windows teamwork mysql c databases system admi...
7,18847370,training team leadership business process impr...
8,19144081,business analysis graphic design user-centered...
9,19826903,linux business analysis c# management sales mi...


In [29]:
# ----------------------------
# Step 6C: Build SOC task "documents" (repeat tasks by weight)
# ----------------------------

# We'll create a SOC document by repeating each task text proportional to its importance
# (simple weighting trick for TF-IDF models)
def make_soc_doc(df_soc_tasks, max_repeat=5):
    parts = []
    for _, r in df_soc_tasks.iterrows():
        task_text = str(r["Task"]).lower().strip()
        if not task_text:
            continue
        # importance_norm in [0,1]; map to repeats 1..max_repeat
        rep = 1 + int(round(r["importance_norm"] * (max_repeat - 1)))
        parts.extend([task_text] * rep)
    return " ".join(parts)

soc_docs = (
    soc_task_req.groupby("soc_code")
    .apply(lambda g: make_soc_doc(g, max_repeat=5))
    .reset_index(name="soc_doc")
)

print("SOC docs:", soc_docs.shape)
display(soc_docs.head(10))


SOC docs: (130, 2)


,soc_code,soc_doc
0,11-1021.00,direct and coordinate activities of businesses...
1,11-2021.00,"identify, develop, or evaluate marketing strat..."
2,11-2022.00,direct and coordinate activities involving sal...
3,11-3011.00,"plan, administer, and control budgets for cont..."
4,11-3021.00,"manage backup, security and user help systems...."
5,11-3051.00,review processing schedules or production orde...
6,11-3061.00,represent companies in negotiating contracts a...
7,11-3071.01,"plan, organize, or manage the work of subordin..."
8,11-3121.00,serve as a link between management and employe...
9,11-3131.00,analyze training needs to develop new training...


In [30]:
!pip install scikit-learn
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------
# Step 7A: Vectorize skill docs + SOC docs together
# ----------------------------

# Keep only positions with a final SOC code and a user id
df_jobs = df_pos[df_pos["soc_code_final"].notna()][["position_id", "user_id", "soc_code_final"]].copy()
df_jobs = df_jobs.rename(columns={"soc_code_final": "soc_code"})

# Merge in docs
df_jobs = df_jobs.merge(user_skill_docs, on="user_id", how="left")
df_jobs = df_jobs.merge(soc_docs, on="soc_code", how="left")

print("Positions to score:", df_jobs.shape[0])
print("Missing skill docs:", df_jobs["skill_doc"].isna().mean())
print("Missing soc docs:", df_jobs["soc_doc"].isna().mean())

# Fill missing with empty strings (score will be ~0)
df_jobs["skill_doc"] = df_jobs["skill_doc"].fillna("")
df_jobs["soc_doc"] = df_jobs["soc_doc"].fillna("")

# Char n-grams are robust to spelling + morphology
vectorizer = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=2,
    max_features=200_000
)

# Fit on combined corpus for shared feature space
corpus = pd.concat([df_jobs["skill_doc"], df_jobs["soc_doc"]], ignore_index=True).tolist()
X = vectorizer.fit_transform(corpus)

n = df_jobs.shape[0]
X_skill = X[:n]
X_soc = X[n:]

# Cosine similarity row-wise (efficient)
# cosine_similarity returns an (n x n) matrix if you pass both; avoid that.
# We'll compute rowwise dot product since vectors are L2-normalized by TF-IDF.
scores = (X_skill.multiply(X_soc)).sum(axis=1)
df_jobs["match_score_tfidf"] = np.asarray(scores).ravel()

print(df_jobs["match_score_tfidf"].describe())

display(df_jobs.head(20))


Positions to score: 3182
Missing skill docs: 0.5424261470773099
Missing soc docs: 0.664676304211188
count    3182.000000
mean        0.030736
std         0.079779
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         0.485199
Name: match_score_tfidf, dtype: float64


,position_id,user_id,soc_code,skill_doc,soc_doc,match_score_tfidf
0,5611932128962179710,840332626,37-3011.00,,,0.000000
1,-3931130892213672808,841151205,25-1191.00,microsoft excel oracle research agile methodol...,,0.000000
2,8045923659602219163,841151205,15-1132.00,microsoft excel oracle research agile methodol...,,0.000000
3,2701527518180934831,841151205,15-1132.00,microsoft excel oracle research agile methodol...,,0.000000
4,-3315325678691742263,841151205,13-1023.00,microsoft excel oracle research agile methodol...,monitor and follow applicable laws and regulat...,0.109650
5,-5108328541936926815,841151205,15-1132.00,microsoft excel oracle research agile methodol...,,0.000000
6,-6242556914544825950,841151205,11-9041.00,microsoft excel oracle research agile methodol...,"confer with management, production, or marketi...",0.084961
7,-5933975230754183227,841403523,11-3011.00,,"plan, administer, and control budgets for cont...",0.000000
8,1527371377900994966,841403523,51-9061.00,,,0.000000
9,3895107283803747500,841403523,51-9061.00,,,0.000000


In [31]:
from pathlib import Path

df_pos = df_pos.merge(
    df_jobs[["position_id", "match_score_tfidf"]],
    on="position_id",
    how="left"
)

print("Scored positions:", df_pos["match_score_tfidf"].notna().mean())
display(df_pos[["jobtitle_raw","soc_code_final","match_type","match_score","match_score_tfidf"]].head(25))

OUT_ARTIFACTS = Path("../data/artifacts")
OUT_ARTIFACTS.mkdir(parents=True, exist_ok=True)

out_csv = OUT_ARTIFACTS / "positions_step7_scored.csv"
df_pos.to_csv(out_csv, index=False)
print("Saved:", out_csv)


Scored positions: 0.9509862522414824


,jobtitle_raw,soc_code_final,match_type,match_score,match_score_tfidf
0,laborer,37-3011.00,exact,100.0,0.000000
1,Teaching Assistant,25-1191.00,exact,100.0,0.000000
2,Software Engineering Intern,15-1132.00,fuzzy,100.0,0.000000
3,Software Engineering Intern,15-1132.00,fuzzy,100.0,0.000000
4,"Analyst, Software Engineer",13-1023.00,fuzzy,100.0,0.109650
5,"Associate, Software Engineer",15-1132.00,exact,100.0,0.000000
6,"Vice President, Software Engineer",11-9041.00,fuzzy,100.0,0.084961
7,Change Control Administrator,11-3011.00,fuzzy,100.0,0.000000
8,Quality System Specialist,51-9061.00,fuzzy,100.0,0.000000
9,Quality Compliance Associate,51-9061.00,fuzzy,100.0,0.000000


Saved: ../data/artifacts/positions_step7_scored.csv


In [32]:
# Rebuild SOC docs from soc_top_tasks (NOT vocab-restricted)
# This should fix the "Missing soc docs: 0.66" issue.

def make_soc_doc_from_top(df_soc_tasks, max_repeat=5):
    parts = []
    for _, r in df_soc_tasks.iterrows():
        task_text = str(r["Task"]).lower().strip()
        if not task_text:
            continue
        rep = 1 + int(round(r["importance_norm"] * (max_repeat - 1)))
        parts.extend([task_text] * rep)
    return " ".join(parts)

soc_docs_full = (
    soc_top_tasks.groupby("soc_code")
    .apply(lambda g: make_soc_doc_from_top(g, max_repeat=5))
    .reset_index(name="soc_doc")
)

print("soc_docs_full shape:", soc_docs_full.shape)
print("Unique SOC in soc_docs_full:", soc_docs_full["soc_code"].nunique())
display(soc_docs_full.head(5))


soc_docs_full shape: (967, 2)
Unique SOC in soc_docs_full: 967


,soc_code,soc_doc
0,11-1011.00,direct or coordinate an organization's financi...
1,11-1011.03,develop or execute strategies to address issue...
2,11-1021.00,"review financial statements, sales or activity..."
3,11-2011.00,plan and prepare advertising and promotional m...
4,11-2021.00,"identify, develop, or evaluate marketing strat..."


In [33]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Rebuild df_jobs cleanly
df_jobs = df_pos[df_pos["soc_code_final"].notna()][["position_id", "user_id", "soc_code_final"]].copy()
df_jobs = df_jobs.rename(columns={"soc_code_final": "soc_code"})

df_jobs = df_jobs.merge(user_skill_docs, on="user_id", how="left")
df_jobs = df_jobs.merge(soc_docs_full, on="soc_code", how="left")  # <-- use full docs

df_jobs["skill_doc"] = df_jobs["skill_doc"].fillna("")
df_jobs["soc_doc"] = df_jobs["soc_doc"].fillna("")

print("Positions to score:", df_jobs.shape[0])
print("Missing skill docs:", df_jobs["skill_doc"].eq("").mean())
print("Missing soc docs:", df_jobs["soc_doc"].eq("").mean())

vectorizer = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=2,
    max_features=200_000
)

corpus = pd.concat([df_jobs["skill_doc"], df_jobs["soc_doc"]], ignore_index=True).tolist()
X = vectorizer.fit_transform(corpus)

n = df_jobs.shape[0]
X_skill = X[:n]
X_soc = X[n:]

df_jobs["match_score_tfidf"] = np.asarray((X_skill.multiply(X_soc)).sum(axis=1)).ravel()

print(df_jobs["match_score_tfidf"].describe())


Positions to score: 3182
Missing skill docs: 0.5424261470773099
Missing soc docs: 0.06505342551854179
count    3182.000000
mean        0.075789
std         0.105684
min         0.000000
25%         0.000000
50%         0.000000
75%         0.141537
max         0.579342
Name: match_score_tfidf, dtype: float64


In [34]:
df_pos = df_pos.drop(columns=["match_score_tfidf"], errors="ignore").merge(
    df_jobs[["position_id", "match_score_tfidf"]],
    on="position_id",
    how="left",
    validate="1:1"
)

print("Scored positions fraction:", df_pos["match_score_tfidf"].notna().mean())


Scored positions fraction: 0.9509862522414824


In [35]:
display(df_pos.sort_values("match_score_tfidf", ascending=False)[
    ["jobtitle_raw","soc_code_final","match_score_tfidf"]
].head(10))

display(df_pos.sort_values("match_score_tfidf", ascending=True)[
    ["jobtitle_raw","soc_code_final","match_score_tfidf"]
].head(10))


,jobtitle_raw,soc_code_final,match_score_tfidf
1386,Security Officer,11-9199.07,0.579342
3071,Manufacturing Process Engineer,17-2199.04,0.530758
3070,Manufacturing & Process Engineer,17-2199.04,0.530758
2138,Senior Staffing Supervisor,13-1071.00,0.499111
1499,Labor Relations Director,11-3121.00,0.472072
1504,Retired Human Resources Professional,11-3121.00,0.472072
1497,Human Resources Manager,11-3121.00,0.472072
1498,Human Resources Manager,11-3121.00,0.472072
1320,Vice President of Operations,11-9021.00,0.469735
1319,Project Executive,11-9021.00,0.469735


,jobtitle_raw,soc_code_final,match_score_tfidf
1323,mecanico,49-3042.00,0.0
2416,Magasinier,11-3071.00,0.0
2011,special customers services,43-4051.00,0.0
2012,Elite Personal Trainer,39-6012.00,0.0
2017,NWKO(NCV),43-9021.00,0.0
2401,Senior Fixed Income Investment Analyst,13-1023.00,0.0
2402,Associate Derivatives Trader,41-3099.01,0.0
2298,Executive Territory Manger,43-6011.00,0.0
1288,Warehouse Manager,11-3071.00,0.0
1289,Service Manager,37-1012.00,0.0


In [36]:
import pandas as pd

# ----------------------------
# Step 7C: Fallback profile docs for users missing skills
# ----------------------------

def norm_token(s):
    if pd.isna(s):
        return ""
    return str(s).strip().lower()

# Build per-user profile doc from job history
profile_parts = []

# job titles (already normalized)
profile_parts.append(
    df_pos.groupby("user_id")["jobtitle_norm"]
    .apply(lambda s: " ".join([x for x in s.dropna().astype(str).tolist() if x]))
    .reset_index()
    .rename(columns={"jobtitle_norm":"profile_title_doc"})
)

# mapped_role
if "mapped_role" in df_pos.columns:
    profile_parts.append(
        df_pos.groupby("user_id")["mapped_role"]
        .apply(lambda s: " ".join([norm_token(x) for x in s.dropna().tolist() if norm_token(x)]))
        .reset_index()
        .rename(columns={"mapped_role":"profile_role_doc"})
    )

# job_category
if "job_category" in df_pos.columns:
    profile_parts.append(
        df_pos.groupby("user_id")["job_category"]
        .apply(lambda s: " ".join([norm_token(x) for x in s.dropna().tolist() if norm_token(x)]))
        .reset_index()
        .rename(columns={"job_category":"profile_cat_doc"})
    )

# Merge parts into one table
user_profile = profile_parts[0]
for part in profile_parts[1:]:
    user_profile = user_profile.merge(part, on="user_id", how="outer")

# Combine into one profile doc
cols = [c for c in user_profile.columns if c != "user_id"]
user_profile["profile_doc"] = user_profile[cols].fillna("").agg(" ".join, axis=1).str.strip()

print("Users with profile docs:", user_profile.shape[0])
display(user_profile.head(10))


Users with profile docs: 1000


,user_id,profile_title_doc,profile_role_doc,profile_cat_doc,profile_doc
0,1350263,general manager licensed optician training spe...,optician sales training training development r...,sales sales admin sales admin admin,general manager licensed optician training spe...
1,1799988,master,captain,sales,master captain sales
2,2802690,legislative,public affairs,marketing,legislative public affairs marketing
3,3031821,behavioral therapist,occupational therapist,scientist,behavioral therapist occupational therapist sc...
4,4217969,ausbildung zum mechatroniker fertigungsassiste...,electronics engineer production technician mec...,engineer engineer engineer engineer engineer e...,ausbildung zum mechatroniker fertigungsassiste...
5,4309523,oracle database administrator oracle database ...,oracle database administrator oracle database ...,engineer engineer engineer,oracle database administrator oracle database ...
6,4496806,cs faculty,teacher,admin,cs faculty teacher admin
7,4497356,site administrator registered polysomnography ...,administrative support clerical support services,admin admin admin,site administrator registered polysomnography ...
8,5776072,quality engineer engineer test,quality assurance engineer software engineer test,engineer engineer engineer,quality engineer engineer test quality assuran...
9,6193890,coordenador de manutencao,maintenance coordinator,engineer,coordenador de manutencao maintenance coordina...


In [37]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# ----------------------------
# Step 7D: Final baseline docs = skill_doc + education_doc + profile_doc
# ----------------------------
# Improvement: Include education text (degree + field) in the user document.
# Rationale: Education is a strong signal for occupational fit — e.g. "Bachelor of
# Computer Science" strongly suggests fit for software/IT SOCs. Including it in
# the TF-IDF document improves task/skill similarity scores.

# Build education docs
edu_path = Path("../data/sample_data_extracted/sample_data/individual_data/individual_user_education.csv")
if edu_path.exists():
    df_edu_docs = pd.read_csv(edu_path)
    def make_edu_text(s):
        if pd.isna(s): return ""
        return str(s).strip().lower()
    
    df_edu_docs["_edu_text"] = (
        df_edu_docs["degree_raw"].apply(make_edu_text) + " " +
        df_edu_docs.get("field_raw", pd.Series([""]* len(df_edu_docs))).apply(make_edu_text)
    ).str.strip()
    
    user_edu_docs = (
        df_edu_docs.groupby("user_id")["_edu_text"]
        .apply(lambda s: " ".join([x for x in s.tolist() if x]))
        .reset_index()
        .rename(columns={"_edu_text": "edu_doc"})
    )
    print("Users with edu docs:", user_edu_docs.shape[0])
else:
    user_edu_docs = pd.DataFrame(columns=["user_id", "edu_doc"])
    print("No education CSV found; skipping edu docs")

# Merge skills + profile + education
user_docs = user_profile.merge(user_skill_docs, on="user_id", how="left")
user_docs = user_docs.merge(user_edu_docs, on="user_id", how="left")

# Choose skill_doc when available, else profile_doc; always append edu_doc
user_docs["skill_doc"] = user_docs["skill_doc"].fillna("")
user_docs["profile_doc"] = user_docs["profile_doc"].fillna("")
user_docs["edu_doc"] = user_docs["edu_doc"].fillna("")

# Primary doc: skills if available, else profile (job history)
user_docs["_primary"] = np.where(
    user_docs["skill_doc"].str.len() > 0,
    user_docs["skill_doc"],
    user_docs["profile_doc"]
)

# Final doc: primary + education (always appended for richer signal)
user_docs["user_doc"] = (user_docs["_primary"] + " " + user_docs["edu_doc"]).str.strip()

print("Users total:", user_docs.shape[0])
print("Users using skills:", (user_docs["skill_doc"].str.len() > 0).mean())
print("Users using fallback profile:", (user_docs["skill_doc"].str.len() == 0).mean())
print("Users with edu appended:", (user_docs["edu_doc"].str.len() > 0).mean())

display(user_docs.head(10))

Users with edu docs: 757
Users total: 1000
Users using skills: 0.339
Users using fallback profile: 0.661
Users with edu appended: 0.692


,user_id,profile_title_doc,profile_role_doc,profile_cat_doc,profile_doc,skill_doc,edu_doc,_primary,user_doc
0,1350263,general manager licensed optician training spe...,optician sales training training development r...,sales sales admin sales admin admin,general manager licensed optician training spe...,retail new store development visual merchandis...,b.a. liberal arts aas ophthalmic science,retail new store development visual merchandis...,retail new store development visual merchandis...
1,1799988,master,captain,sales,master captain sales,,,master captain sales,master captain sales
2,2802690,legislative,public affairs,marketing,legislative public affairs marketing,microsoft office public speaking nonprofits so...,effective teacher training biology/biological ...,microsoft office public speaking nonprofits so...,microsoft office public speaking nonprofits so...
3,3031821,behavioral therapist,occupational therapist,scientist,behavioral therapist occupational therapist sc...,,licensure licensed practical/vocational nurse ...,behavioral therapist occupational therapist sc...,behavioral therapist occupational therapist sc...
4,4217969,ausbildung zum mechatroniker fertigungsassiste...,electronics engineer production technician mec...,engineer engineer engineer engineer engineer e...,ausbildung zum mechatroniker fertigungsassiste...,,fachhochschulriefe maschinenbau und fertigungt...,ausbildung zum mechatroniker fertigungsassiste...,ausbildung zum mechatroniker fertigungsassiste...
5,4309523,oracle database administrator oracle database ...,oracle database administrator oracle database ...,engineer engineer engineer,oracle database administrator oracle database ...,,master's degree information technology,oracle database administrator oracle database ...,oracle database administrator oracle database ...
6,4496806,cs faculty,teacher,admin,cs faculty teacher admin,,,cs faculty teacher admin,cs faculty teacher admin
7,4497356,site administrator registered polysomnography ...,administrative support clerical support services,admin admin admin,site administrator registered polysomnography ...,,associate of science (a.s.),site administrator registered polysomnography ...,site administrator registered polysomnography ...
8,5776072,quality engineer engineer test,quality assurance engineer software engineer test,engineer engineer engineer,quality engineer engineer test quality assuran...,web services software project management quali...,be computer science,web services software project management quali...,web services software project management quali...
9,6193890,coordenador de manutencao,maintenance coordinator,engineer,coordenador de manutencao maintenance coordina...,,,coordenador de manutencao maintenance coordina...,coordenador de manutencao maintenance coordina...


In [38]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Rebuild df_jobs cleanly
df_jobs2 = df_pos[df_pos["soc_code_final"].notna()][["position_id", "user_id", "soc_code_final"]].copy()
df_jobs2 = df_jobs2.rename(columns={"soc_code_final": "soc_code"})

df_jobs2 = df_jobs2.merge(user_docs[["user_id", "user_doc"]], on="user_id", how="left")
df_jobs2 = df_jobs2.merge(soc_docs_full, on="soc_code", how="left")

df_jobs2["user_doc"] = df_jobs2["user_doc"].fillna("")
df_jobs2["soc_doc"] = df_jobs2["soc_doc"].fillna("")

print("Positions to score:", df_jobs2.shape[0])
print("Missing user docs:", df_jobs2["user_doc"].eq("").mean())
print("Missing soc docs:", df_jobs2["soc_doc"].eq("").mean())

vectorizer = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=2,
    max_features=200_000
)

corpus = pd.concat([df_jobs2["user_doc"], df_jobs2["soc_doc"]], ignore_index=True).tolist()
X = vectorizer.fit_transform(corpus)

n = df_jobs2.shape[0]
X_user = X[:n]
X_soc = X[n:]

df_jobs2["match_score_tfidf_v2"] = np.asarray((X_user.multiply(X_soc)).sum(axis=1)).ravel()

print(df_jobs2["match_score_tfidf_v2"].describe())


Positions to score: 3182
Missing user docs: 0.0
Missing soc docs: 0.06505342551854179
count    3182.000000
mean        0.133715
std         0.090664
min         0.000000
25%         0.073419
50%         0.117471
75%         0.177576
max         0.573011
Name: match_score_tfidf_v2, dtype: float64


In [39]:
df_pos = df_pos.drop(columns=["match_score_tfidf_v2"], errors="ignore").merge(
    df_jobs2[["position_id", "match_score_tfidf_v2"]],
    on="position_id",
    how="left",
    validate="1:1"
)

print("Scored positions fraction:", df_pos["match_score_tfidf_v2"].notna().mean())


Scored positions fraction: 0.9509862522414824


In [40]:
display(df_pos.sort_values("match_score_tfidf_v2", ascending=False)[
    ["jobtitle_raw","soc_code_final","match_score_tfidf_v2"]
].head(10))

display(df_pos.sort_values("match_score_tfidf_v2", ascending=True)[
    ["jobtitle_raw","soc_code_final","match_score_tfidf_v2"]
].head(10))


,jobtitle_raw,soc_code_final,match_score_tfidf_v2
1386,Security Officer,11-9199.07,0.573011
1901,IT Project Manager II,15-1199.09,0.557179
3070,Manufacturing & Process Engineer,17-2199.04,0.500897
3071,Manufacturing Process Engineer,17-2199.04,0.500897
1319,Project Executive,11-9021.00,0.494182
1320,Vice President of Operations,11-9021.00,0.494182
2138,Senior Staffing Supervisor,13-1071.00,0.484848
1499,Labor Relations Director,11-3121.00,0.459194
1498,Human Resources Manager,11-3121.00,0.459194
1497,Human Resources Manager,11-3121.00,0.459194


,jobtitle_raw,soc_code_final,match_score_tfidf_v2
3325,Gerente de Marketing,11-2011.01,0.0
3324,Auxiliar de marketing,11-2011.01,0.0
849,Account Officer,21-1029.00,0.0
850,Senior Account Officer,21-1029.00,0.0
756,Accountant,13-2011.00,0.0
757,Auditor,13-2011.00,0.0
806,Volunteer Coordinator,43-9199.00,0.0
808,Shift Manager,11-9199.00,0.0
3227,Warehouse Operations Specialist,13-1199.00,0.0
925,Trainee,11-9199.00,0.0


In [41]:
from pathlib import Path
import pandas as pd

REQ_BASE = Path("../data/sample_data_extracted/sample_data/occupation_requirements")
ete_path = REQ_BASE / "Education, Training, and Experience.xlsx"
ete_cat_path = REQ_BASE / "Education, Training, and Experience Categories.xlsx"

df_ete = pd.read_excel(ete_path)
df_ete_cat = pd.read_excel(ete_cat_path)

print("ETE:", df_ete.shape)
print("ETE Categories:", df_ete_cat.shape)

print("\nETE columns:")
print(list(df_ete.columns))

print("\nETE Categories columns:")
print(list(df_ete_cat.columns))

display(df_ete.head(10))
display(df_ete_cat.head(25))


ETE: (39866, 15)
ETE Categories: (41, 6)

ETE columns:
['O*NET-SOC Code', 'Title', 'Element ID', 'Element Name', 'Scale ID', 'Scale Name', 'Category', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound', 'Upper CI Bound', 'Recommend Suppress', 'Date', 'Domain Source']

ETE Categories columns:
['Element ID', 'Element Name', 'Scale ID', 'Scale Name', 'Category', 'Category Description']


,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Category,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Date,Domain Source
0,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),1.0,0.00,27,0.00,NaN,NaN,N,07/2014,Incumbent
1,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),2.0,0.00,27,0.00,NaN,NaN,N,07/2014,Incumbent
2,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),3.0,0.00,27,0.00,NaN,NaN,N,07/2014,Incumbent
3,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),4.0,6.05,27,4.43,1.28,24.19,N,07/2014,Incumbent
4,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),5.0,4.23,27,4.27,0.50,27.80,N,07/2014,Incumbent
5,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),6.0,21.61,27,9.79,7.76,47.47,N,07/2014,Incumbent
6,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),7.0,13.66,27,8.42,3.52,40.70,N,07/2014,Incumbent
7,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),8.0,25.83,27,11.34,9.35,54.03,N,07/2014,Incumbent
8,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),9.0,19.23,27,16.21,2.71,67.05,Y,07/2014,Incumbent
9,11-1011.00,Chief Executives,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),10.0,0.00,27,0.00,NaN,NaN,N,07/2014,Incumbent


,Element ID,Element Name,Scale ID,Scale Name,Category,Category Description
0,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),1,Less than a High School Diploma
1,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),2,High School Diploma - or the equivalent (for e...
2,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),3,Post-Secondary Certificate - awarded for train...
3,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),4,Some College Courses
4,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),5,Associate's Degree (or other 2-year degree)
5,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),6,Bachelor's Degree
6,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),7,Post-Baccalaureate Certificate - awarded for c...
7,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),8,Master's Degree
8,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),9,Post-Master's Certificate - awarded for comple...
9,2.D.1,Required Level of Education,RL,Required Level Of Education (Categories 1-12),10,First Professional Degree - awarded for comple...


In [42]:
# ----------------------------
# Step 8B: Build SOC requirement levels
# ----------------------------

# Keep only the scales we care about
KEEP_SCALES = ["RL", "RW", "PT"]

df_req = df_ete[df_ete["Scale ID"].isin(KEEP_SCALES)].copy()

df_req = df_req[[
    "O*NET-SOC Code",
    "Scale ID",
    "Category",
    "Data Value"
]].rename(columns={
    "O*NET-SOC Code":"soc_code",
    "Category":"category",
    "Data Value":"pct"
})

# Remove missing
df_req = df_req.dropna(subset=["soc_code","category","pct"])

# Weighted average category per SOC per scale
soc_req_levels = (
    df_req.groupby(["soc_code","Scale ID"])
    .apply(lambda g: (g["category"] * g["pct"]).sum() / g["pct"].sum())
    .reset_index(name="required_level")
)

display(soc_req_levels.head(20))

print("SOC coverage:", soc_req_levels["soc_code"].nunique())


,soc_code,Scale ID,required_level
0,11-1011.00,PT,4.716200
1,11-1011.00,RL,7.536300
2,11-1011.00,RW,9.247875
3,11-1011.03,PT,4.346300
4,11-1011.03,RL,6.923108
5,11-1011.03,RW,8.884488
6,11-1021.00,PT,3.881488
7,11-1021.00,RL,4.915508
8,11-1021.00,RW,8.106211
9,11-2011.00,PT,3.133300


SOC coverage: 967


In [43]:
soc_req_wide = soc_req_levels.pivot(index="soc_code", columns="Scale ID", values="required_level").reset_index()

soc_req_wide = soc_req_wide.rename(columns={
    "RL":"req_edu_level",
    "RW":"req_exp_level",
    "PT":"req_train_level"
})

display(soc_req_wide.head(20))


Scale ID,soc_code,req_train_level,req_edu_level,req_exp_level
0,11-1011.00,4.716200,7.536300,9.247875
1,11-1011.03,4.346300,6.923108,8.884488
2,11-1021.00,3.881488,4.915508,8.106211
3,11-2011.00,3.133300,5.207000,7.442300
4,11-2021.00,2.432400,6.680500,8.281244
5,11-2022.00,3.695730,6.043596,7.956600
6,11-2031.00,2.504150,6.102390,7.751525
7,11-3011.00,3.293471,3.963204,6.972494
8,11-3021.00,3.496850,5.550855,8.875688
9,11-3031.01,2.566357,7.066600,9.166917


In [44]:
from pathlib import Path
import pandas as pd

edu_path = Path("../data/sample_data_extracted/sample_data/individual_data/individual_user_education.csv")
df_edu = pd.read_csv(edu_path)

print(df_edu.shape)
print(df_edu.columns)
display(df_edu.head(20))


(1406, 6)
Index(['user_id', 'school', 'startdate', 'enddate', 'degree_raw', 'field_raw'], dtype='str')


,user_id,school,startdate,enddate,degree_raw,field_raw
0,80200835,University of Tennessee-Knoxville,1990-01-01,1995-01-31,NaN,NaN
1,554516311,Texas McCombs School of Business,2022-01-01,2022-01-01,Post Graduate Degree - Data Science and Busine...,NaN
2,515451089,Campbell University,1992-01-01,1995-01-01,NaN,NaN
3,256939147,University of Illinois at Urbana-Champaign,2006-01-01,2012-01-01,Ph.D.,Chemistry
4,8904119,Pontificia Universidad Javeriana,2000-01-01,2005-01-01,Bachelor's degree,NaN
5,380268833,"Pui Shing Catholic Secondary School, Hong Kong",1997-01-01,2002-01-01,High School,Secondary School Courses
6,995451723,University of Information Technology,2014-01-01,2019-01-31,Bachelor's degree,Computer Science
7,684165615,University of Minnesota-Twin Cities,NaN,NaN,Bachelor of Science (BS),"Business Administration and Management, General"
8,170452190,Lycée IHSSANE,2015-01-01,2015-01-01,Baccalauréat,Mention Très Bien
9,498293119,Lancaster Bible College | Capital Seminary & G...,2012-01-01,2016-01-01,Bachelor of Arts - BA,Spiritual Formation & Discipleship


In [45]:
import pandas as pd
import numpy as np
import re

# ----------------------------
# Step 8E: Map degrees to RL scale (1–12)
# ----------------------------
# O*NET Required Level (RL) scale:
#  1 = Less than High School
#  2 = High School Diploma / GED
#  3 = Post-Secondary Certificate
#  5 = Associate's Degree
#  6 = Bachelor's Degree
#  7 = Post-Baccalaureate Certificate
#  8 = Master's Degree
#  9 = Post-Master's Certificate
# 10 = First Professional Degree (JD, MD, DDS, etc.)
# 11 = Doctoral Degree (PhD, EdD, etc.)
# 12 = Post-Doctoral Training
#
# Assumption: We map degree_raw text to the best matching RL category via regex.
# Ambiguous degrees default to NaN rather than guessing wrong.

def map_degree_to_rl(s):
    if pd.isna(s):
        return np.nan
    
    s = str(s).lower().strip()
    if not s:
        return np.nan

    # Post-doctoral
    if re.search(r'post[- ]?doc|postdoc|habilitation', s):
        return 12

    # PhD / Doctoral
    if re.search(r'ph\.?d|doctor|doctoral|dphil|d\.phil|edd|ed\.d', s):
        return 11

    # First Professional Degree (JD, MD, DDS, PharmD, etc.)
    if re.search(r'\bj\.?d\b|juris doctor|law degree|llb|ll\.b|ll\.m'
                 r'|\bm\.?d\b|doctor of medicine|medical degree'
                 r'|\bd\.?d\.?s\b|dentist|dent|pharm\.?d|doctor of pharmacy'
                 r'|\bd\.?v\.?m\b|veterinar', s):
        return 10

    # Post-Master's Certificate
    if re.search(r'post[- ]?master|specialist|ed\.?s', s):
        return 9

    # Master's
    if re.search(r'master|msc|m\.s\.|m\.a\.|mba|m\.b\.a'
                 r'|ma |ms |meng|m\.eng|mres|mphil|m\.phil'
                 r'|post ?grad|graduate degree|maestr', s):
        return 8

    # Post-Baccalaureate Certificate
    if re.search(r'post[- ]?bac|graduate certificate|graduate diploma', s):
        return 7

    # Bachelor's
    if re.search(r'bachelor|bsc|b\.s\.|ba |b\.a\.|beng|b\.eng|bba|bs '
                 r'|undergraduate|licenciatura|laurea|4[- ]year', s):
        return 6

    # Associate
    if re.search(r'associate|aa |a\.a\.|as |a\.s\.|2[- ]year|community college', s):
        return 5

    # Post-Secondary Certificate / vocational
    if re.search(r'certificate|certification|diploma|technical|trade|vocational|apprentice', s):
        return 3

    # High school / GED
    if re.search(r'high school|secondary school|baccalaur|ged|'
                 r'general education|school diploma', s):
        return 2

    # Some education but below HS
    if re.search(r'some college|some school|no degree|incomplete|dropout|drop out', s):
        return 1

    return np.nan

df_edu["edu_level_rl"] = df_edu["degree_raw"].apply(map_degree_to_rl)

print("Mapped education rate:", df_edu["edu_level_rl"].notna().mean())
print("\nEducation level distribution:")
print(df_edu["edu_level_rl"].value_counts().sort_index())
display(df_edu[["degree_raw","edu_level_rl"]].drop_duplicates().sort_values("edu_level_rl").head(40))

Mapped education rate: 0.5846372688477952

Education level distribution:
edu_level_rl
1.0       1
2.0      24
3.0      55
5.0      63
6.0     416
8.0     214
9.0       4
10.0     11
11.0     34
Name: count, dtype: int64


,degree_raw,edu_level_rl
145,No Degree,1.0
1195,Baccalauréat mention très bien,2.0
1166,Baccalauréat général,2.0
1149,Baccalauréat sciences et technologies du manag...,2.0
924,High School Degree,2.0
710,Première année du baccalauréat,2.0
587,General Education,2.0
968,"French Baccalauréat Scientist, Maths",2.0
228,Primary And Secondary School,2.0
5,High School,2.0


In [46]:
user_edu = (
    df_edu.groupby("user_id")["edu_level_rl"]
    .max()
    .reset_index()
    .rename(columns={"edu_level_rl":"user_edu_level"})
)

print("Users with education:", user_edu["user_edu_level"].notna().mean())
display(user_edu.head(20))


Users with education: 0.6882430647291942


,user_id,user_edu_level
0,1350263,6.0
1,1799988,NaN
2,2802690,NaN
3,3031821,NaN
4,4217969,8.0
5,4309523,8.0
6,4496806,NaN
7,4497356,5.0
8,5776072,NaN
9,6327198,8.0


In [47]:
# Merge SOC requirements
df_pos = df_pos.merge(soc_req_wide, left_on="soc_code_final", right_on="soc_code", how="left")

# Merge worker education
df_pos = df_pos.merge(user_edu, on="user_id", how="left")

# Education gap
df_pos["edu_gap"] = df_pos["user_edu_level"] - df_pos["req_edu_level"]

# Convert to match score (0–1)
# 0 = far underqualified, 1 = meets/exceeds
df_pos["edu_match_score"] = np.clip(1 + df_pos["edu_gap"]/4, 0, 1)

print(df_pos[["user_edu_level","req_edu_level","edu_gap","edu_match_score"]].describe())
display(df_pos[["jobtitle_raw","user_edu_level","req_edu_level","edu_match_score"]].head(20))


       user_edu_level  req_edu_level      edu_gap  edu_match_score
count     2167.000000    2975.000000  1950.000000      1950.000000
mean         6.934933       5.396143     1.479415         0.936268
std          1.784801       1.686630     2.108078         0.170555
min          1.000000       1.401000    -5.536300         0.000000
25%          6.000000       4.489000     0.121688         1.000000
50%          6.000000       5.580542     1.332500         1.000000
75%          8.000000       6.304470     2.806950         1.000000
max         11.000000      11.605500     9.310931         1.000000


,jobtitle_raw,user_edu_level,req_edu_level,edu_match_score
0,laborer,NaN,2.746700,NaN
1,Teaching Assistant,6.0,6.908191,0.772952
2,Software Engineering Intern,6.0,6.238176,0.940456
3,Software Engineering Intern,6.0,6.238176,0.940456
4,"Analyst, Software Engineer",6.0,5.695530,1.000000
5,"Associate, Software Engineer",6.0,6.238176,0.940456
6,"Vice President, Software Engineer",6.0,6.715072,0.821232
7,Change Control Administrator,8.0,3.963204,1.000000
8,Quality System Specialist,8.0,2.539446,1.000000
9,Quality Compliance Associate,8.0,2.539446,1.000000


In [48]:
import pandas as pd
import numpy as np

# ----------------------------
# Step 9A: Compute cumulative experience per job spell
# ----------------------------

df_exp = df_pos.copy()

# Ensure datetime
df_exp["startdate"] = pd.to_datetime(df_exp["startdate"], errors="coerce")
df_exp["enddate"] = pd.to_datetime(df_exp["enddate"], errors="coerce")

# If enddate missing assume ongoing
today = pd.Timestamp.today()
df_exp["enddate"] = df_exp["enddate"].fillna(today)

# Duration in years
df_exp["duration_years"] = (df_exp["enddate"] - df_exp["startdate"]).dt.days / 365.25

# Sort per user
df_exp = df_exp.sort_values(["user_id","startdate"])

# cumulative experience BEFORE the job
df_exp["prior_experience_years"] = (
    df_exp.groupby("user_id")["duration_years"]
    .cumsum()
    .shift(fill_value=0)
)

display(df_exp[["user_id","jobtitle_raw","duration_years","prior_experience_years"]].head(20))


,user_id,jobtitle_raw,duration_years,prior_experience_years
3249,1350263,General Manager / Licensed Optician,6.247775,0.000000
3250,1350263,Training Specialist,1.333333,6.247775
3251,1350263,Senior Learning Specialist,5.333333,7.581109
3252,1350263,Retail Operations Manager,3.252567,12.914442
3253,1350263,Leadership Development Manager,5.286790,16.167009
3254,1350263,Senior Manager of People Development,3.624914,21.453799
3255,1799988,Master,28.958248,25.078713
456,2802690,Legislative Assistant,15.121150,28.958248
457,3031821,Behavioral Therapist,7.036277,15.121150
2598,4217969,Ausbildung zum Mechatroniker,2.830938,7.036277


In [57]:
# ----------------------------
# Step 9B: Convert years to RW category
# ----------------------------

def years_to_rw(y):
    if pd.isna(y): return np.nan
    if y <= 0: return 1
    if y <= 1/12: return 2
    if y <= 3/12: return 3
    if y <= 6/12: return 4
    if y <= 1: return 5
    if y <= 2: return 6
    if y <= 4: return 7
    if y <= 6: return 8
    if y <= 8: return 9
    if y <= 10: return 10
    return 11

df_exp["user_exp_level"] = df_exp["prior_experience_years"].apply(years_to_rw)

display(df_exp[["prior_experience_years","user_exp_level"]].head(20))


,prior_experience_years,user_exp_level
0,0.000000,1
1,6.247775,9
2,7.581109,9
3,12.914442,11
4,16.167009,11
5,21.453799,11
6,25.078713,11
7,28.958248,11
8,15.121150,11
9,7.036277,9


In [60]:
# Merge required experience
# Drop columns that already exist from the earlier soc_req_wide merge on df_pos
cols_to_merge = ["soc_code", "req_exp_level", "req_train_level"]
df_exp = df_exp.drop(columns=[c for c in cols_to_merge if c in df_exp.columns], errors="ignore")

df_exp = df_exp.merge(soc_req_wide[cols_to_merge],
                      left_on="soc_code_final", right_on="soc_code", how="left")

# Experience gap
df_exp["exp_gap"] = df_exp["user_exp_level"] - df_exp["req_exp_level"]

df_exp["exp_match_score"] = np.clip(1 + df_exp["exp_gap"]/4, 0, 1)

# Training gap (simpler: just compare levels)
df_exp["train_gap"] = df_exp["user_exp_level"] - df_exp["req_train_level"]
df_exp["train_match_score"] = np.clip(1 + df_exp["train_gap"]/4, 0, 1)

display(df_exp[["user_exp_level","req_exp_level","exp_match_score"]].describe())

,user_exp_level,req_exp_level,exp_match_score
count,3346.000000,2975.000000,2975.000000
mean,8.881949,6.668384,0.915680
std,2.291491,1.727394,0.215829
min,1.000000,1.250200,0.000000
25%,7.000000,5.591459,1.000000
50%,9.000000,7.117100,1.000000
75%,11.000000,8.038496,1.000000
max,11.000000,9.623738,1.000000


In [62]:
import json

# ----------------------------
# Step 10: Data-driven composite weights
# ----------------------------
# Instead of hardcoding arbitrary weights (e.g. 0.60/0.20/0.15/0.05), we derive
# them from the data using ANOVA eta-squared: the fraction of total variance in
# each component that is explained by SOC group membership.
#
# Rationale: components that better discriminate between occupations should
# contribute more to the final score.  A component with high between-SOC
# variance relative to total variance is a stronger occupational-fit signal.

# ── Bring experience/training scores from df_exp back into df_pos ──
# df_exp is a copy of df_pos with extra columns computed in Steps 9A/9B.
exp_cols = ["exp_match_score", "train_match_score", "user_exp_level",
            "prior_experience_years", "duration_years", "exp_gap", "train_gap"]
exp_merge_cols = [c for c in exp_cols if c in df_exp.columns and c not in df_pos.columns]
if exp_merge_cols:
    df_pos = df_pos.merge(
        df_exp[["user_id", "soc_code_final", "startdate"] + exp_merge_cols].drop_duplicates(),
        on=["user_id", "soc_code_final", "startdate"],
        how="left",
    )
    print(f"Merged {exp_merge_cols} from df_exp → df_pos")

components = ["match_score_tfidf_v2", "edu_match_score", "exp_match_score", "train_match_score"]
component_labels = ["tfidf", "edu", "exp", "train"]

# Fill missing values conservatively (same as before)
df_pos["edu_match_score"] = df_pos["edu_match_score"].fillna(0.5)
df_pos["exp_match_score"] = df_pos["exp_match_score"].fillna(0.5)
df_pos["train_match_score"] = df_pos["train_match_score"].fillna(0.5)

# Only use rows with a mapped SOC code for the eta-squared calculation
df_eta = df_pos.dropna(subset=["soc_code_final"]).copy()

eta_sq = {}
for comp in components:
    grand_mean = df_eta[comp].mean()
    
    # SS_between: sum of (group_mean - grand_mean)^2 * group_size
    group_stats = df_eta.groupby("soc_code_final")[comp].agg(["mean", "count"])
    ss_between = ((group_stats["mean"] - grand_mean) ** 2 * group_stats["count"]).sum()
    
    # SS_total: sum of (value - grand_mean)^2
    ss_total = ((df_eta[comp] - grand_mean) ** 2).sum()
    
    eta_sq[comp] = ss_between / ss_total if ss_total > 0 else 0.0

print("--- Eta-squared (between-SOC variance / total variance) ---")
for comp, eta in eta_sq.items():
    bar = "█" * int(eta * 50)
    print(f"  {comp:25s}  η² = {eta:.4f}  {bar}")

# Normalize to weights summing to 1
raw_weights = np.array([eta_sq[c] for c in components])
weights = raw_weights / raw_weights.sum()

# Build weight dict
weight_names = dict(zip(component_labels, [round(float(w), 4) for w in weights]))
print(f"\n--- Derived weights ---")
for label, w in weight_names.items():
    print(f"  w_{label} = {w:.4f}")
print(f"  Sum = {sum(weight_names.values()):.4f}")

# Assign to variables for use below
w_tfidf, w_edu, w_exp, w_train = weights

# Compute final composite score with data-driven weights
df_pos["match_score_final"] = (
    w_tfidf * df_pos["match_score_tfidf_v2"] +
    w_edu   * df_pos["edu_match_score"] +
    w_exp   * df_pos["exp_match_score"] +
    w_train * df_pos["train_match_score"]
)

print(f"\nmatch_score_final stats:")
print(df_pos["match_score_final"].describe())

# Save weights to JSON for downstream use (demo, validation, etc.)
weights_out = {
    "method": "eta_squared_anova",
    "description": "Weights derived from between-SOC variance ratio (eta-squared). Higher eta-squared means the component better discriminates between occupations.",
    "weights": {
        "match_score_tfidf_v2": weight_names["tfidf"],
        "edu_match_score": weight_names["edu"],
        "exp_match_score": weight_names["exp"],
        "train_match_score": weight_names["train"],
    },
    "eta_squared": {comp: round(float(eta_sq[comp]), 6) for comp in components},
    "n_soc_groups": int(df_eta["soc_code_final"].nunique()),
    "n_positions": len(df_eta),
}
weights_path = Path("../data/derived_weights.json")
weights_path.write_text(json.dumps(weights_out, indent=2), encoding="utf-8")
print(f"\nSaved weights to {weights_path}")

display(df_pos.sort_values("match_score_final", ascending=False)[
    ["jobtitle_raw","soc_code_final","match_score_final"]
].head(10))

display(df_pos.sort_values("match_score_final", ascending=True)[
    ["jobtitle_raw","soc_code_final","match_score_final"]
].head(10))

Merged ['exp_match_score', 'train_match_score', 'user_exp_level', 'prior_experience_years', 'duration_years', 'exp_gap', 'train_gap'] from df_exp → df_pos
--- Eta-squared (between-SOC variance / total variance) ---
  match_score_tfidf_v2       η² = 0.5840  █████████████████████████████
  edu_match_score            η² = 0.3100  ███████████████
  exp_match_score            η² = 0.3377  ████████████████
  train_match_score          η² = 0.7717  ██████████████████████████████████████

--- Derived weights ---
  w_tfidf = 0.2915
  w_edu = 0.1547
  w_exp = 0.1686
  w_train = 0.3852
  Sum = 1.0000

match_score_final stats:
count    3218.000000
mean        0.677214
std         0.110294
min         0.087791
25%         0.652587
50%         0.706160
75%         0.743026
max         0.854518
Name: match_score_final, dtype: float64

Saved weights to ../data/derived_weights.json


,jobtitle_raw,soc_code_final,match_score_final
3110,Manufacturing & Process Engineer,17-2199.04,0.854518
3111,Manufacturing Process Engineer,17-2199.04,0.854518
1333,Project Executive,11-9021.00,0.852560
1334,Vice President of Operations,11-9021.00,0.852560
2158,Senior Staffing Supervisor,13-1071.00,0.849840
1773,HR Legal,13-1071.00,0.836383
1778,HR Generalist,13-1071.00,0.836383
1777,HR Specialist,13-1071.00,0.836383
1779,HR Business Partner,13-1071.00,0.836383
986,Staff Musician,27-2041.04,0.832255


,jobtitle_raw,soc_code_final,match_score_final
2085,Conducteur de travaux,11-3051.02,0.087791
1852,Software Web Developer,11-9021.00,0.093233
3109,Agente de servicios al cliente,13-1041.04,0.129382
848,Member,25-1122.00,0.166882
2538,Style Consultant,13-1111.00,0.173644
838,Global Credit Manager,43-4041.01,0.188415
3289,General Manager / Licensed Optician,11-9021.00,0.202016
2403,Class A Driver,53-3032.00,0.248317
937,Intern,11-9041.00,0.260559
1252,Intern,11-9041.00,0.262727


In [63]:

# ----------------------------
# Step 11A: Qualification direction
# ----------------------------

# Overqualified = worker exceeds requirements
df_pos["overqualified"] = (
    (df_pos["edu_gap"] > 1) &
    (df_pos["exp_match_score"] >= 1)
)

# Underqualified = lacks minimum requirements
df_pos["underqualified"] = (
    (df_pos["edu_gap"] < -1) |
    (df_pos["exp_match_score"] < 0.5)
)

# Pure skill mismatch (qualified but bad task match)
df_pos["skill_mismatch"] = (
    (df_pos["match_score_tfidf_v2"] < 0.15) &
    (~df_pos["underqualified"])
)

# Good match
df_pos["good_match"] = (
    (df_pos["match_score_final"] >= 0.50) &
    (~df_pos["underqualified"])
)

print(df_pos[["good_match","overqualified","underqualified","skill_mismatch"]].mean())


good_match        0.773479
overqualified     0.272593
underqualified    0.115771
skill_mismatch    0.540756
dtype: float64


In [64]:
def classify_match(row):
    if row["underqualified"]:
        return "underqualified"
    if row["overqualified"]:
        return "overqualified"
    if row["skill_mismatch"]:
        return "mismatch"
    if row["good_match"]:
        return "good_match"
    return "weak_match"

df_pos["match_category"] = df_pos.apply(classify_match, axis=1)

print(df_pos["match_category"].value_counts(normalize=True))


match_category
mismatch          0.363851
overqualified     0.272593
good_match        0.198169
underqualified    0.115771
weak_match        0.049616
Name: proportion, dtype: float64


In [65]:
# ----------------------------
# Step 12: Mapping confidence score
# ----------------------------
# A confidence score (0-1) indicating how reliable the title->SOC mapping is.
# This helps downstream consumers know when to trust the match score vs. treat it cautiously.
#
# Factors:
#   - match_type: exact=1.0, fuzzy=scaled by match_score, role_fallback=0.3
#   - non-English penalty: reduce confidence for non-English titles (fuzzy is unreliable)
#   - missing SOC: confidence=0

def compute_mapping_confidence(row):
    if pd.isna(row.get("soc_code_final")):
        return 0.0

    mt = str(row.get("match_type", "")).lower()
    ms = row.get("match_score", 0)
    if pd.isna(ms):
        ms = 0

    if mt == "exact":
        conf = 1.0
    elif mt == "fuzzy":
        # Scale fuzzy score from [threshold, 100] -> [0.5, 0.95]
        conf = 0.5 + 0.45 * max(0, min(1, (ms - 85) / 15))
    else:
        # role fallback or unknown
        conf = 0.3

    # Non-English penalty
    if row.get("title_non_english_flag", False):
        conf *= 0.7

    return round(conf, 3)

df_pos["mapping_confidence"] = df_pos.apply(compute_mapping_confidence, axis=1)

print("Mapping confidence distribution:")
print(df_pos["mapping_confidence"].describe())
print("\nBy match_type:")
print(df_pos.groupby("match_type")["mapping_confidence"].agg(["mean", "median", "count"]))

Mapping confidence distribution:
count    3386.000000
mean        0.861012
std         0.278750
min         0.000000
25%         0.950000
50%         0.950000
75%         1.000000
max         1.000000
Name: mapping_confidence, dtype: float64

By match_type:
                mean  median  count
match_type                         
exact       1.000000    1.00   1354
fuzzy       0.940607    0.95   1573


In [66]:
# ----------------------------
# Merge user_doc + soc_doc into df_pos for export
# ----------------------------

# 1) user_doc: built in Step 7D on the user_docs DataFrame
if "user_doc" not in df_pos.columns:
    df_pos = df_pos.merge(
        user_docs[["user_id", "user_doc"]],
        on="user_id",
        how="left"
    )
    print("Merged user_doc into df_pos")

# 2) soc_doc: from soc_docs_full (keyed on soc_code)
#    Drop any prior soc_doc columns to avoid _x/_y suffixes
for c in ["soc_doc", "soc_doc_x", "soc_doc_y"]:
    if c in df_pos.columns:
        df_pos = df_pos.drop(columns=[c])

soc_docs_full2 = soc_docs_full.copy()
soc_docs_full2["soc_code_final"] = soc_docs_full2["soc_code"]
soc_docs_full2 = soc_docs_full2[["soc_code_final", "soc_doc"]].drop_duplicates("soc_code_final")
df_pos = df_pos.merge(soc_docs_full2, on="soc_code_final", how="left")

# Fill NaNs
df_pos["user_doc"] = df_pos["user_doc"].fillna("")
df_pos["soc_doc"]  = df_pos["soc_doc"].fillna("")

print(f"user_doc present: {(df_pos['user_doc'].str.len() > 0).mean():.1%}")
print(f"soc_doc present:  {(df_pos['soc_doc'].str.len() > 0).mean():.1%}")
print(f"Columns: {len(df_pos.columns)}")

Merged user_doc into df_pos
user_doc present: 100.0%
soc_doc present:  88.9%
Columns: 61


In [67]:
# (Merged into cell above — this cell is intentionally empty)
# The user_doc + soc_doc merge and export are handled above.

In [68]:
# Ensure mapping_confidence is present
if "mapping_confidence" not in df_pos.columns:
    df_pos["mapping_confidence"] = np.where(df_pos["soc_code_final"].notna(), 0.5, 0.0)

df_pos.to_csv("../data/final_job_match_dataset.csv", index=False)
print("Exported ../data/final_job_match_dataset.csv")
print("Columns:", list(df_pos.columns))
print("Shape:", df_pos.shape)

Exported ../data/final_job_match_dataset.csv
Columns: ['user_id', 'position_id', 'company_raw', 'company_linkedin_url', 'company_cleaned', 'region', 'country', 'state', 'metro_area', 'startdate', 'enddate', 'jobtitle_raw', 'mapped_role', 'job_category', 'seniority', 'salary', 'rn', 'rcid', 'company_name', 'ultimate_parent_rcid', 'ultimate_parent_company_name', 'jobtitle_norm', 'soc_candidates_x', 'match_type_x', 'match_score_x', 'soc_code_top1', 'soc_candidates_y', 'match_type_y', 'match_score_y', 'soc_candidates', 'match_type', 'match_score', 'title_non_english_flag', 'soc_accept', 'soc_code_final', 'soc_code_fallback_role', 'match_score_tfidf', 'match_score_tfidf_v2', 'soc_code', 'req_train_level', 'req_edu_level', 'req_exp_level', 'user_edu_level', 'edu_gap', 'edu_match_score', 'exp_match_score', 'train_match_score', 'user_exp_level', 'prior_experience_years', 'duration_years', 'exp_gap', 'train_gap', 'match_score_final', 'overqualified', 'underqualified', 'skill_mismatch', 'good_ma